# Imports

In [ ]:
import sys
sys.path.insert(1, "..")
from collections import namedtuple
import dill
from helpFunctions import get_string_representation_of_kernel as gsr
from helpFunctions import get_full_kernels_in_kernel_expression
from globalParams import options
import gpytorch
import itertools
import matplotlib.colors as mcolors
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
from metrics import calculate_AIC as AIC, calculate_BIC as BIC, calculate_laplace as Laplace, NestedSampling as Nested, log_normalized_prior, prior_distribution
import numpy as np
import os
import pandas as pd
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct
from pygranso.private.getNvar import getNvarTorch
import re
import torch

# Seed

In [ ]:
torch.manual_seed(42)


# Data generation

## 2D

In [ ]:
def periodic_1D(X):
    """
    $\sin(x_0)$
    """
    return torch.sin(X[:,0])

def periodic_2D(X):
    """
    $\sin(x_0) \cdot \sin(x_1)$
    """
    return torch.sin(X[:,0]) * torch.sin(X[:,1])

def parabola_1D(X):
    """
    $x_0^2$
    """
    return X[:,0]**2

def parabola_2D(X):
    """
    $x_0^2 \cdot x_1^2$
    """
    return X[:,0]**2 + X[:,1]**2

def product(X):
    """
    $x_0 \cdot x_1$
    """
    return X[:,0] * X[:,1]

def periodic_sum(X):
    """
    $\sin(x_0 + x_1)$
    """
    return torch.sin(X[:,0] + X[:,1])

def periodic_sincos(X):
    """
    $\sin(x_0) \cdot \cos(x_1)$
    """
    return torch.sin(X[:,0]) * torch.cos(X[:,1])

def linear_1D(X):
    """
    $x_0$
    """
    return X[:,0]

def linear_2D(X):
    """
    $x_0 + x_1$
    """
    return X[:,0]+X[:,1]


# Helper functions

In [ ]:
hyperparameter_limits = {"RBFKernel": {"lengthscale": [1e-3,5]},
                         "MaternKernel": {"lengthscale": [1e-3,1]},
                         "LinearKernel": {"variance": [1e-4,1]},
                         "AffineKernel": {"variance": [1e-4,1]},
                         "RQKernel": {"lengthscale": [1e-3,1],
                                      "alpha": [1e-3,1]},
                         "CosineKernel": {"period_length": [1e-3,3]},
                         "PeriodicKernel": {"lengthscale": [1e-3,5],
                                            "period_length": [1e-3,10]},
                         "ScaleKernel": {"outputscale": [1e-3,10]},
                         "Noise": [1e-3, 1],
                         "MyPeriodKernel":{"period_length": [1e-3,3]}}

def random_reinit(model, logarithmic=False):
    #print("Random reparameterization")
    #print("old parameters: ", list(model.named_parameters()))
    model_params = model.parameters()
    relevant_hyper_limits = list()
    relevant_hyper_limits.append(hyperparameter_limits["Noise"])
    kernel_name_list = [kernel for kernel in get_full_kernels_in_kernel_expression(model.covar_module)]
    for kernel in kernel_name_list:
        for param_name in hyperparameter_limits[kernel]:
            relevant_hyper_limits.append(hyperparameter_limits[kernel][param_name])

    for i, (param, limit) in enumerate(zip(model_params, relevant_hyper_limits)):
        param_name = limit
        if logarithmic:
            new_param_value = torch.rand_like(param)* (torch.log(torch.tensor(limit[1])) - torch.log(torch.tensor(limit[0]))) + torch.log(torch.tensor(limit[0]))
        else:
            new_param_value = torch.rand_like(param)* (limit[1] - limit[0]) + limit[0]
        param.data = new_param_value
    #print("new parameters: ", list(model.named_parameters()))


def fixed_reinit(model, parameters: torch.tensor) -> None:
    for i, (param, value) in enumerate(zip(model.parameters(), parameters)):
        param.data = torch.full_like(param.data, value)

# Define the training loop
def optimize_hyperparameters(model, likelihood, **kwargs):
    """
    find optimal hyperparameters either by BO or by starting from random initial values multiple times, using an optimizer every time
    and then returning the best result
    """

    # I think this is very ugly to define the class inside the training function and then use a parameter from the function within the class scope. But we all need to make sacrifices...
    # Original class taken from https://ncvx.org/examples/A1_rosenbrock.html
    class HaltLog:
        def __init__(self):
            pass

        def haltLog(self, iteration, x, penaltyfn_parts, d,get_BFGS_state_fn, H_regularized,
                    ls_evals, alpha, n_gradients, stat_vec, stat_val, fallback_level):

            # DON'T CHANGE THIS
            # increment the index/count
            self.index += 1

            # EXAMPLE:
            # store history of x iterates in a preallocated cell array
            self.x_iterates[restart].append(x)
            self.neg_loss[restart].append(penaltyfn_parts.f)
            self.tv[restart].append(penaltyfn_parts.tv)
            self.hessians[restart].append(get_BFGS_state_fn())

            # keep this false unless you want to implement a custom termination
            # condition
            halt = False
            return halt

        # Once PyGRANSO has run, you may call this function to get retreive all
        # the logging data stored in the shared variables, which is populated
        # by haltLog being called on every iteration of PyGRANSO.
        def getLog(self):
            # EXAMPLE
            # return x_iterates, trimmed to correct size
            log = pygransoStruct()
            log.x        = self.x_iterates
            log.neg_loss = self.neg_loss
            log.tv       = self.tv
            log.hessians = self.hessians
            #log = pygransoStruct()
            #log.x   = self.x_iterates[0:self.index]
            #log.f   = self.f[0:self.index]
            #log.tv  = self.tv[0:self.index]
            #log.hessians  = self.hessians[0:self.index]
            return log

        def makeHaltLogFunctions(self, restarts=1):
            # don't change these lambda functions
            halt_log_fn = lambda iteration, x, penaltyfn_parts, d,get_BFGS_state_fn, H_regularized, ls_evals, alpha, n_gradients, stat_vec, stat_val, fallback_level: self.haltLog(iteration, x, penaltyfn_parts, d,get_BFGS_state_fn, H_regularized, ls_evals, alpha, n_gradients, stat_vec, stat_val, fallback_level)

            get_log_fn = lambda : self.getLog()

            # Make your shared variables here to store PyGRANSO history data
            # EXAMPLE - store history of iterates x_0,x_1,...,x_k

            # restart the index and empty the log
            self.index       = 0
            self.x_iterates  = [list() for _ in range(restarts)]
            self.neg_loss    = [list() for _ in range(restarts)]
            self.tv          = [list() for _ in range(restarts)]
            self.hessians    = [list() for _ in range(restarts)]

            # Only modify the body of logIterate(), not its name or arguments.
            # Store whatever data you wish from the current PyGRANSO iteration info,
            # given by the input arguments, into shared variables of
            # makeHaltLogFunctions, so that this data can be retrieved after PyGRANSO
            # has been terminated.
            #
            # DESCRIPTION OF INPUT ARGUMENTS
            #   iter                current iteration number
            #   x                   current iterate x
            #   penaltyfn_parts     struct containing the following
            #       OBJECTIVE AND CONSTRAINTS VALUES
            #       .f              objective value at x
            #       .f_grad         objective gradient at x
            #       .ci             inequality constraint at x
            #       .ci_grad        inequality gradient at x
            #       .ce             equality constraint at x
            #       .ce_grad        equality gradient at x
            #       TOTAL VIOLATION VALUES (inf norm, for determining feasibiliy)
            #       .tvi            total violation of inequality constraints at x
            #       .tve            total violation of equality constraints at x
            #       .tv             total violation of all constraints at x
            #       TOTAL VIOLATION VALUES (one norm, for L1 penalty function)
            #       .tvi_l1         total violation of inequality constraints at x
            #       .tvi_l1_grad    its gradient
            #       .tve_l1         total violation of equality constraints at x
            #       .tve_l1_grad    its gradient
            #       .tv_l1          total violation of all constraints at x
            #       .tv_l1_grad     its gradient
            #       PENALTY FUNCTION VALUES
            #       .p              penalty function value at x
            #       .p_grad         penalty function gradient at x
            #       .mu             current value of the penalty parameter
            #       .feasible_to_tol logical indicating whether x is feasible
            #   d                   search direction
            #   get_BFGS_state_fn   function handle to get the (L)BFGS state data
            #                       FULL MEMORY:
            #                       - returns BFGS inverse Hessian approximation
            #                       LIMITED MEMORY:
            #                       - returns a struct with current L-BFGS state:
            #                           .S          matrix of the BFGS s vectors
            #                           .Y          matrix of the BFGS y vectors
            #                           .rho        row vector of the 1/sty values
            #                           .gamma      H0 scaling factor
            #   H_regularized       regularized version of H
            #                       [] if no regularization was applied to H
            #   fn_evals            number of function evaluations incurred during
            #                       this iteration
            #   alpha               size of accepted size
            #   n_gradients         number of previous gradients used for computing
            #                       the termination QP
            #   stat_vec            stationarity measure vector
            #   stat_val            approximate value of stationarity:
            #                           norm(stat_vec)
            #                       gradients (result of termination QP)
            #   fallback_level      number of strategy needed for a successful step
            #                       to be taken.  See bfgssqpOptionsAdvanced.
            #
            # OUTPUT ARGUMENT
            #   halt                set this to true if you wish optimization to
            #                       be halted at the current iterate.  This can be
            #                       used to create a custom termination condition,
            return [halt_log_fn, get_log_fn]


    random_restarts = kwargs.get("random_restarts", options["training"]["restarts"]+1)
    uninformed = kwargs.get("uninformed", False)
    logarithmic_reinit = kwargs.get("logarithmic_reinit", False)
    train_log = [list() for _ in range(random_restarts)]

    """
    # The call that comes from GRANSO
    user_halt = halt_log_fn(0, x, self.penaltyfn_at_x, np.zeros((n,1)),
                                        get_bfgs_state_fn, H_QP,
                                        1, 0, 1, stat_vec, self.stat_val, 0          )
    """
    def log_fnct(*args):
        train_log[restart].append((args[1], args[2].neg_loss))
        return False 


    train_x = kwargs.get("X", model.train_inputs)
    train_y = kwargs.get("Y", model.train_targets)
    MAP = kwargs.get("MAP", True)
    double_precision = kwargs.get("double_precision", False)

    # Set up the likelihood and model
    #likelihood = gpytorch.likelihoods.GaussianLikelihood()
    #model = GPModel(train_x, train_y, likelihood)

    # Define the negative log likelihood
    mll_fkt = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # Set up the PyGRANSO optimizer
    opts = pygransoStruct()
    opts.torch_device = torch.device('cpu')
    nvar = getNvarTorch(model.parameters())
    opts.x0 = torch.nn.utils.parameters_to_vector(model.parameters()).detach().reshape(nvar,1)
    opts.opt_tol = float(1e-10)
    #opts.limited_mem_size = int(100)
    opts.limited_mem_size = 0
    opts.globalAD = True
    opts.double_precision = double_precision
    opts.quadprog_info_msg = False
    opts.print_level = int(0)
    opts.halt_on_linesearch_bracket = False
    mHLF_obj = HaltLog()
    [halt_log_fn, get_log_fn] = mHLF_obj.makeHaltLogFunctions(restarts=random_restarts)

    #  Set PyGRANSO's logging function in opts
    opts.halt_log_fn = halt_log_fn

    # Define the objective function
    def objective_function(model):
        output = model(train_x)
        try:
            # TODO PyGRANSO dying is a severe problem. as it literally exits the program instead of raising an error
            # negative scaled MLL
            loss = -mll_fkt(output, train_y)
        except Exception as E:
            print("LOG ERROR: Severe PyGRANSO issue. Loss is inf+0")
            loss = torch.tensor(np.inf, requires_grad=True) + torch.tensor(0)
        if MAP:
            # log_normalized_prior is in metrics.py 
            log_p = log_normalized_prior(model, uninformed=uninformed)
            # negative scaled MAP
            loss -= log_p
        #print(f"LOG: {loss}")
        return [loss, None, None]

    best_model_state_dict = model.state_dict()
    best_likelihood_state_dict = likelihood.state_dict()

    best_f = np.inf
    for restart in range(random_restarts):
        print("---")
        print("start parameters: ", opts.x0)
        # Train the model using PyGRANSO
        try:
            soln = pygranso(var_spec=model, combined_fn=objective_function, user_opts=opts)
            print(f"Restart {restart} : trained parameters: {list(model.named_parameters())}")
        except Exception as e:
            print(e)
            import pdb
            pdb.set_trace()
            pass

        if soln.final.f < best_f:
            print(f"LOG: Found new best solution: {soln.final.f}")
            best_f = soln.final.f
            best_model_state_dict = model.state_dict()
            best_likelihood_state_dict = likelihood.state_dict()
        random_reinit(model, logarithmic=logarithmic_reinit)
        opts.x0 = torch.nn.utils.parameters_to_vector(model.parameters()).detach().reshape(nvar,1)

    model.load_state_dict(best_model_state_dict)
    likelihood.load_state_dict(best_likelihood_state_dict)
    print(f"----")
    print(f"Final best parameters: {list(model.named_parameters())} w. loss: {best_f} (smaller=better)")
    print(f"----")

    loss = -mll_fkt(model(train_x), train_y)
    #print(f"LOG: Final MLL: {loss}")
    if MAP:
        log_p = log_normalized_prior(model, uninformed=uninformed)
        loss -= log_p
        #print(f"LOG: Final MAP: {loss}")

    #print(f"post training (best): {list(model.named_parameters())} w. loss: {soln.best.f}")
    #print(f"post training (final): {list(model.named_parameters())} w. loss: {soln.final.f}")
    
    #print(torch.autograd.grad(loss, [p for p in model.parameters()], retain_graph=True, create_graph=True, allow_unused=True))
    # Return the trained model
    return loss, model, likelihood, get_log_fn()


def get_std_points(mu, K):
    x, y = np.mgrid[-3:3:.1, -3:3:.1]
    L = np.linalg.cholesky(K)

    data = np.dstack((x, y))

    # Drawing the unit circle
    # x^2 + y^2 = 1
    precision = 50
    unit_x = torch.cat([torch.linspace(-1, 1, precision), torch.linspace(-1, 1, precision)])
    unit_y = torch.cat([torch.sqrt(1 - torch.linspace(-1, 1, precision)**2), -torch.sqrt(1 - torch.linspace(-1, 1, precision)**2)])

    new_unit_x = list()
    new_unit_y = list()

    for tx, ty in zip(unit_x, unit_y):
        res = np.array([tx, ty]) @ L
        new_unit_x.append(mu[0] + 2*res[0])
        new_unit_y.append(mu[1] + 2*res[1])
    return new_unit_x, new_unit_y


# Find all points inside the confidence ellipse
def percentage_inside_ellipse(mu, K, points, sigma_level=2):
    L = np.linalg.cholesky(K)
    threshold = sigma_level ** 2
    count = 0
    for point in points:
        res = np.array(point - mu) @ np.linalg.inv(L)
        if res @ res <= threshold:
            count += 1
    return count / len(points)


def log_dill(data, filename):
    with open(filename, 'wb') as f:
        dill.dump(data, f)

In [ ]:
# https://www.sfu.ca/sasdoc/sashtml/iml/chap11/sect8.htm
# Also https://en.wikipedia.org/wiki/Finite_difference_coefficient
def finite_difference_second_derivative_GP_neg_unscaled_map(model, likelihood, train_x, train_y, uninformed=False, h_i_step=1e-3, h_j_step=1e-3, h_i_vec=[0.0, 0.0, 0.0], h_j_vec=[0.0, 0.0, 0.0]):
    curr_params = torch.tensor(list(model.parameters()))
    mll_fkt = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    h_i = h_i_step * torch.tensor(h_i_vec)
    h_j = h_j_step * torch.tensor(h_j_vec)

    fixed_reinit(model, curr_params+h_i + h_j)
    f_plus = (-mll_fkt(model(train_x), train_y) - log_normalized_prior(model, uninformed=uninformed))*len(*model.train_inputs)

    fixed_reinit(model, curr_params+h_i - h_j)
    f1 = (-mll_fkt(model(train_x), train_y) - log_normalized_prior(model, uninformed=uninformed))*len(*model.train_inputs)
    fixed_reinit(model, curr_params-h_i + h_j)
    f2 = (-mll_fkt(model(train_x), train_y) - log_normalized_prior(model, uninformed=uninformed))*len(*model.train_inputs)

    fixed_reinit(model, curr_params - h_i - h_j)
    f_minus = (-mll_fkt(model(train_x), train_y) - log_normalized_prior(model, uninformed=uninformed))*len(*model.train_inputs)

    # Reverse model reparameterization
    fixed_reinit(model, curr_params)

    return (f_plus - f1 - f2 + f_minus) / (4*h_i_step*h_j_step)

In [ ]:
def finite_difference_hessian(model, likelihood, num_params, train_x, train_y, uninformed=False, h_i_step=5e-2, h_j_step=5e-2):
    hessian_finite_differences_neg_unscaled_map = np.zeros((num_params, num_params))
    for i, j in itertools.product(range(num_params), range(num_params)):
        h_i_vec = np.zeros(num_params)
        h_j_vec = np.zeros(num_params)
        h_i_vec[i] = 1.0
        h_j_vec[j] = 1.0
        hessian_finite_differences_neg_unscaled_map[i][j] = finite_difference_second_derivative_GP_neg_unscaled_map(model, likelihood, train_x, train_y, uninformed=uninformed, h_i_step=h_i_step, h_j_step=h_j_step, h_i_vec=h_i_vec, h_j_vec=h_j_vec)
    return hessian_finite_differences_neg_unscaled_map

# Plotting functions

## Training

In [ ]:
def plot_parameter_progression(parameter_progression, losses=None, xlabel=None, ylabel=None, fig=None, ax=None, xdim=0, ydim=1, display_figure=True, return_figure=False, title_add=""):
    if not (fig and ax):
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    
    # Reverse colormap by using "viridis_r"
    cmap = plt.cm.viridis_r  

    # Define a normalization based on loss values
    norm = None
    if losses is not None:
        norm = mcolors.Normalize(vmin=min(losses[1:]), vmax=max(losses[1:]))  # Keep vmin and vmax as usual
    
    for i in range(1, len(parameter_progression)):
        if losses is None:
            colormap = cmap(i / len(parameter_progression))  # Use reversed colormap
        else:
            colormap = cmap(norm(losses[i]))  # Normalize and apply reversed colormap
        
        ax.plot([parameter_progression[i-1][xdim], parameter_progression[i][xdim]], 
                [parameter_progression[i-1][ydim], parameter_progression[i][ydim]], 
                color=colormap)

        ax.annotate("", xy=(parameter_progression[i][xdim], parameter_progression[i][ydim]), 
                    xytext=(parameter_progression[i-1][xdim], parameter_progression[i-1][ydim]),  
                    arrowprops=dict(arrowstyle="->", color=colormap, lw=2))
    
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(f"Parameter progression {title_add}")

    # Add colorbar with reversed colormap
    if losses is not None:
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)  # Use reversed colormap here
        sm.set_array([])  # Required for colorbar
        fig.colorbar(sm, ax=ax, label="Loss value (higher = worse)")

    if return_figure:
        return fig, ax
    if display_figure:
        plt.show()
    return None, None


## Nested

In [ ]:

# Stolen from https://matplotlib.org/stable/gallery/statistics/confidence_ellipse.html
def confidence_ellipse(mu, K, ax, n_std=2.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The Axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """

    cov = K
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensional dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the standard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = mu[0] 

    # calculating the standard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = mu[1]

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

def nested_sampling_plot(model, model_evidence_log, xdim=0, ydim=1, filter_type="none", std_filter=None, show_last_num=None, return_figure=False, title_add="", fig=None, ax=None, display_figure=True, plot_mll_opt=False, mll_opt_params=None, plot_lap=False, Lap0_logs=None, LapAIC_logs=None, LapBIC_logs=None, lap_colors = ["r", "pink", "black"], Lap_hess=None):

    if not (fig and ax):
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))

    with open(f"{model_evidence_log['res file']}", "rb") as f:
        res = dill.load(f)
    
    # Plot the actual figure
    param_names = [l[0] for l in list(model.named_parameters())]

    # Find the best value and the corresponding hyperparameters
    best_idx = np.argmax(res.logl)
    best_hyperparameters = res.samples[best_idx]

    # Do an outlier cleanup based on the std_filter or the last "show_last_num" samples
    if show_last_num is not None:
        if type(show_last_num) is int:
            # Find value of "show_last_num" sample
            filter_val = sorted(res.logl, reverse=True)[show_last_num]
            mask = res.logl > filter_val
        elif type(show_last_num) is float:
            # Raise an error if show_last_num is not between 0 and 1
            if show_last_num < 0 or show_last_num > 1:
                raise ValueError("show_last_num must be between 0 and 1")
            # assume that it is a percentage of the total samples
            filter_val = sorted(res.logl, reverse=True)[int(len(res.logl)*show_last_num)]
            mask = res.logl > filter_val

    # Do an outlier cleanup on res.logz
    if std_filter is None and not filter_type == "none" and not show_last_num is None:
        raise ValueError("Cannot use both filter_type and show_last_num at the same time")
    if show_last_num is None and not std_filter is None:
        logz_std = np.std(res.logl)
        if filter_type == "max":
            mask = res.logl >= max(res.logl)+std_filter*logz_std
        elif filter_type == "mean":
            raise NotImplementedError("This filter type is not implemented yet")
            logz_mean = np.mean(res.logz)
            mask = np.all(logz_mean - abs(std_filter) * logz_std <= res.logz <= logz_mean + abs(std_filter) * logz_std)
        elif filter_type == "none":
            mask = res.logl == res.logl



    likelihood_surface_scatter = ax.scatter(res.samples[:,xdim][mask], res.samples[:,ydim][mask], c=res.logl[mask], s=3)
    # Best found hyperparameters
    ax.scatter(best_hyperparameters[xdim], best_hyperparameters[ydim], c="r", s=10)

    if plot_mll_opt and not mll_opt_params is None:
        ax.scatter(mll_opt_params[xdim], mll_opt_params[ydim], c="black", s=10)
        # Add a small text beside the point saying "MLL"
        ax.text(mll_opt_params[xdim], mll_opt_params[ydim], "MLL", fontsize=12, color="black", verticalalignment='center', horizontalalignment='right')
    
    coverages = list()
    if plot_lap:
        # Plot the Laplace levels
        for lap_log, lap_color in zip([Lap0_logs, LapAIC_logs, LapBIC_logs], lap_colors):
            if lap_log is None:
                continue
            lap_param_mu = lap_log["parameter values"]
            # Wait a minute, isn't the Hessian the inverse of the covariance matrix? Yes, see Murphy PML 1 eq. (7.228)
            lap_param_cov_matr = torch.linalg.inv(lap_log["corrected Hessian"])
            # Calculate the amount of samples that are covered by the 1 sigma and 2 sigma interval based on the lap_mu and lap_cov values
            lap_2_sig_coverage = percentage_inside_ellipse(lap_param_mu.flatten().numpy(), lap_param_cov_matr.numpy(), res.samples[mask])
            coverages.append(lap_2_sig_coverage)
            #ax.scatter(lap_param_mu[xdim], lap_param_mu[ydim], c="b", s=10)

            # Plot the std points
            lap_mu_filtered = lap_param_mu.numpy()[[xdim, ydim]] 
            lap_cov_filtered = lap_param_cov_matr.numpy()[[xdim, ydim]][:,[xdim, ydim]]
            #lap_var_ellipse_x, lap_var_ellipse_y = get_std_points(lap_mu_filtered.flatten(), lap_cov_filtered)
            #plt.scatter(lap_var_ellipse_x, lap_var_ellipse_y, c="b", s=1)
            confidence_ellipse(lap_mu_filtered, lap_cov_filtered, ax, n_std=2, edgecolor=lap_color, lw=1)
        if not Lap_hess is None:
            # It can happen that the Hessian is not invertible, in that case drawing the confidence ellipse is not possible
            try:
                lap_param_cov_matr = torch.linalg.inv(Lap_hess)
                lap_2_sig_coverage = percentage_inside_ellipse(lap_param_mu.flatten().numpy(), lap_param_cov_matr.numpy(), res.samples[mask])

                coverages.append(lap_2_sig_coverage)

                lap_mu_filtered = lap_param_mu.numpy()[[xdim, ydim]] 
                lap_cov_filtered = lap_param_cov_matr.numpy()[[xdim, ydim]][:,[xdim, ydim]]
                confidence_ellipse(lap_mu_filtered, lap_cov_filtered, ax, n_std=2, edgecolor="black", lw=1)
            except Exception as e:
                print(e)


    if show_last_num is not None:
        ax.set_title(f"#Samples: {sum(res.ncall)}; {coverages[0]*100:.0f}% inside 2 sigma\n{show_last_num} best accepted samples")
    elif not std_filter is None:
        ax.set_title(f"#Samples: {sum(res.ncall)}; {coverages[0]*100:.0f}% inside 2 sigma\n{filter_type}: {std_filter:.0e}")
    else:
        ax.set_title(f"#Samples: {sum(res.ncall)}; {coverages[0]*100:.0f}% inside 2 sigma\n")
    ax.set_xlabel(param_names[xdim])

    plt.colorbar(likelihood_surface_scatter)

    if return_figure:
        return fig, ax
    if display_figure:
        plt.show()
    return None, None





def outlier_cleanup(values, mode="none", filter=None):
    """
    Outlier cleanup function for the nested sampling results.

    Parameters
    ----------
    values : array-like
        The values to be filtered.
    mode : str
        The mode of filtering. Can be "none", "max", "min", "mean", or "value".
    filter : float or int
        The filter value. If mode is "max" or "min", it can be a percentage (float) or a number of samples (int). 
        If mode is "mean", it is the number of standard deviations from the mean. If mode is "value", it is the threshold value.

    """

    # Filter modes: "none", "max", "mean", "value"
    # "none": There is no filtering, just pass everything
    # "max": Only display the highest "std_filter" samples. If "std_filter" is float, use percent, if int, use the number of samples
    # "min": Only display the smallest "std_filter" samples. If "std_filter" is float, use percent, if int, use the number of samples
    # "mean": Only display the samples that are within "std_filter" standard deviations of the mean
    # "value": Only display the samples that are greater than "std_filter"

    masked=None

    if mode == "none":
        masked = values == values
    elif mode == "min":
        if type(filter) is float:
            # Raise an error if show_last_num is not between 0 and 1
            if filter < 0 or filter > 1:
                raise ValueError("filter must be between 0 and 1")
            # assume that it is a percentage of the total samples
            filter_val = sorted(values)[int(len(values)*filter)]
            masked = values < filter_val
        elif type(filter) is int:
            # Find value of "show_last_num" sample
            filter_val = sorted(values)[filter]
            masked = values < filter_val 
    elif mode == "max":
        if type(filter) is float:
            # Raise an error if show_last_num is not between 0 and 1
            if filter < 0 or filter > 1:
                raise ValueError("filter must be between 0 and 1")
            # assume that it is a percentage of the total samples
            filter_val = sorted(values, reverse=True)[int(len(values)*filter)]
            masked = values > filter_val
        elif type(filter) is int:
            # Find value of "show_last_num" sample
            filter_val = sorted(values, reverse=True)[filter]
            masked = values > filter_val 
    elif mode == "mean":
        raise NotImplementedError("This filter type is not implemented yet")
        logz_mean = np.mean(res.logz)
        masked = np.all(logz_mean - abs(filter) * logz_std <= res.logz <= logz_mean + abs(std_filter) * logz_std)
    elif mode == "value":
        masked = values > filter
    return masked



# More general version of the nested sampling plotting function
def scatter_function_visualization(positions, values, fig=None, ax=None, axis_names=None, show_best=False, xdim=0, ydim=1, filter_mode="none", std_filter=None, show_last_num=None, return_figure=False, title_add="", display_figure=True):
   
    if not (fig and ax):
        fig, ax = plt.subplots(1, 1, figsize=(8, 6)) 

    mask = outlier_cleanup(values, filter_mode, std_filter)
    
    filtered_x_dim = positions[:,xdim][mask]
    filtered_y_dim = positions[:,ydim][mask]
    filtered_colors = values[mask]
    likelihood_surface_scatter = ax.scatter(filtered_x_dim, filtered_y_dim, c=filtered_colors, s=3)

    if show_best:
        # Find the best value and the corresponding hyperparameters
        best_idx = np.argmax(values)
        best_hyperparameters = positions[best_idx]
        # Best found hyperparameters
        ax.scatter(best_hyperparameters[xdim], best_hyperparameters[ydim], c="r", s=10)

    if show_last_num is not None:
        ax.set_title(f"#Samples: {len(values)}; {show_last_num} best accepted samples")
    elif not std_filter is None:
        ax.set_title(f"#Samples: {len(values)}; {filter_mode}: {std_filter:.0e}")
    else:
        ax.set_title(f"#Samples: {len(values)}; ")
    ax.set_xlabel(axis_names[0])
    ax.set_ylabel(axis_names[1])

    plt.colorbar(likelihood_surface_scatter)

    if return_figure:
        return fig, ax
    if display_figure:
        plt.show()
    return None, None



## 1D

In [ ]:

def plot_data(X, Y, return_figure=False, title_add="", figure=None, ax=None, display_figure=True):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111)
    ax.plot(X.numpy(), Y.numpy(), 'k.')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title(f"Data {title_add}")
    if not return_figure and display_figure:
        plt.show()
    else:
        return fig, ax

def plot_model(model, likelihood, X, Y, display_figure=True, return_figure=False, figure=None,
               ax=None, loss_val=None, loss_type = None):
    interval_length = torch.max(X) - torch.min(X)
    shift = interval_length * options["plotting"]["border_ratio"]
    test_x = torch.linspace(torch.min(
        X) - shift, torch.max(X) + shift, options["plotting"]["sample_points"])

    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        observed_pred = likelihood(model(test_x))

    with torch.no_grad():
        if not (figure and ax):
            figure, ax = plt.subplots(1, 1, figsize=(8, 6))

        lower, upper = observed_pred.confidence_region()
        ax.plot(X.numpy(), Y.numpy(), 'k.', zorder=2)
        ax.plot(test_x.numpy(), observed_pred.mean.numpy(), color="b", zorder=3)
        amount_of_gradient_steps = 30
        alpha_min = 0.05
        alpha_max = 0.8
        alpha = (alpha_max-alpha_min)/amount_of_gradient_steps
        c = ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(
        ), alpha=alpha+alpha_min, zorder=1).get_facecolor()

        for i in range(1, amount_of_gradient_steps):
            ax.fill_between(test_x.numpy(), (lower+(i/amount_of_gradient_steps)*(upper-lower)).numpy(),
                            (upper-(i/amount_of_gradient_steps)*(upper-lower)).numpy(), alpha=alpha, color=c, zorder=1)
        if options["plotting"]["legend"]:
            ax.plot([], [], 'k.', label="Data")
            ax.plot([], [], 'b', label="Mean")
            ax.plot([], [], color=c, alpha=1.0, label="Confidence")
            ax.legend(loc="upper left")
        ax.set_xlabel("Normalized Input")
        ax.set_ylabel("Normalized Output")
        ax.set_title(f"{loss_type}: {loss_val:.2f}")
    if not return_figure and display_figure:
        plt.show()
    else:
        return figure, ax


In [ ]:
# make a nested sampling plot, but multiply each point with the respective prior likelihood, thus generating the Posterior surface instead of the likelihood surface


def posterior_surface_plot(model, model_evidence_log, xdim=0, ydim=1, filter_type="none", std_filter=None, show_last_num=None, return_figure=False, title_add="", fig=None, ax=None, display_figure=True, plot_mll_opt=False, mll_opt_params=None, plot_lap=False, Lap0_logs=None, LapAIC_logs=None, LapBIC_logs=None, lap_colors = ["r", "pink", "black"], Lap_hess=None, uninformed=False):

    if not (fig and ax):
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))

    with open(f"{model_evidence_log['res file']}", "rb") as f:
        res = dill.load(f)
    
    # Plot the actual figure
    param_names = [l[0] for l in list(model.named_parameters())]


    # rescale the "res.samples" by the prior likelihood
    prior_mu, prior_sigma = prior_distribution(model=model, uninformed=uninformed)
    prior = torch.distributions.MultivariateNormal(prior_mu.t(), prior_sigma)
    posterior_samples = np.array([logl+prior.log_prob(torch.tensor(sample)).item() for sample, logl in zip(res.samples, res.logl)])

    # Find the best value and the corresponding hyperparameters
    best_idx = np.argmax(posterior_samples)
    best_hyperparameters = res.samples[best_idx]

    # Do an outlier cleanup based on the std_filter or the last "show_last_num" samples
    if show_last_num is not None:
        if type(show_last_num) is int:
            # Find value of "show_last_num" sample
            filter_val = sorted(posterior_samples, reverse=True)[show_last_num]
            mask = posterior_samples > filter_val
        elif type(show_last_num) is float:
            # Raise an error if show_last_num is not between 0 and 1
            if show_last_num < 0 or show_last_num > 1:
                raise ValueError("show_last_num must be between 0 and 1")
            # assume that it is a percentage of the total samples
            filter_val = sorted(posterior_samples, reverse=True)[int(len(posterior_samples)*show_last_num)]
            mask = posterior_samples > filter_val

    # Do an outlier cleanup on res.logz
    if std_filter is None and not filter_type == "none" and not show_last_num is None:
        raise ValueError("Cannot use both filter_type and show_last_num at the same time")
    if show_last_num is None and not std_filter is None:
        logz_std = np.std(posterior_samples)
        if filter_type == "max":
            mask = posterior_samples >= max(posterior_samples)+std_filter*logz_std
        elif filter_type == "mean":
            raise NotImplementedError("This filter type is not implemented yet")
            logz_mean = np.mean(res.logz)
            mask = np.all(logz_mean - abs(std_filter) * logz_std <= res.logz <= logz_mean + abs(std_filter) * logz_std)
        elif filter_type == "none":
            mask = posterior_samples == posterior_samples


    posterior_surface_scatter = ax.scatter(res.samples[:,xdim][mask], res.samples[:,ydim][mask], c=posterior_samples[mask], s=3)
    # Best found hyperparameters
    ax.scatter(best_hyperparameters[xdim], best_hyperparameters[ydim], c="r", s=10)

    if plot_mll_opt and not mll_opt_params is None:
        ax.scatter(mll_opt_params[xdim], mll_opt_params[ydim], c="black", s=10)
        # Add a small text beside the point saying "MLL"
        ax.text(mll_opt_params[xdim], mll_opt_params[ydim], "MLL", fontsize=12, color="black", verticalalignment='center', horizontalalignment='right')
    
    coverages = list()
    if plot_lap:
        # Plot the Laplace levels
        for lap_log, lap_color in zip([Lap0_logs, LapAIC_logs, LapBIC_logs], lap_colors):
            if lap_log is None:
                continue
            lap_param_mu = lap_log["parameter values"]
            # Wait a minute, isn't the Hessian the inverse of the covariance matrix? Yes, see Murphy PML 1 eq. (7.228)
            lap_param_cov_matr = torch.linalg.inv(lap_log["corrected Hessian"])
            # Calculate the amount of samples that are covered by the 1 sigma and 2 sigma interval based on the lap_mu and lap_cov values
            lap_2_sig_coverage = percentage_inside_ellipse(lap_param_mu.flatten().numpy(), lap_param_cov_matr.numpy(), res.samples[mask])
            coverages.append(lap_2_sig_coverage)
            #ax.scatter(lap_param_mu[xdim], lap_param_mu[ydim], c="b", s=10)

            # Plot the std points
            lap_mu_filtered = lap_param_mu.numpy()[[xdim, ydim]] 
            lap_cov_filtered = lap_param_cov_matr.numpy()[[xdim, ydim]][:,[xdim, ydim]]
            #lap_var_ellipse_x, lap_var_ellipse_y = get_std_points(lap_mu_filtered.flatten(), lap_cov_filtered)
            #plt.scatter(lap_var_ellipse_x, lap_var_ellipse_y, c="b", s=1)
            confidence_ellipse(lap_mu_filtered, lap_cov_filtered, ax, n_std=2, edgecolor=lap_color, lw=1)
        if not Lap_hess is None:
            # It can happen that the Hessian is not invertible, in that case drawing the confidence ellipse is not possible
            try:
                lap_param_cov_matr = torch.linalg.inv(Lap_hess)
                lap_2_sig_coverage = percentage_inside_ellipse(lap_param_mu.flatten().numpy(), lap_param_cov_matr.numpy(), res.samples[mask])

                coverages.append(lap_2_sig_coverage)

                lap_mu_filtered = lap_param_mu.numpy()[[xdim, ydim]] 
                lap_cov_filtered = lap_param_cov_matr.numpy()[[xdim, ydim]][:,[xdim, ydim]]
                confidence_ellipse(lap_mu_filtered, lap_cov_filtered, ax, n_std=2, edgecolor="black", lw=1)
            except Exception as e:
                print(e)


    if show_last_num is not None:
        ax.set_title(f"#Samples: {sum(res.ncall)}; {coverages[0]*100:.0f}% inside 2 sigma\n{show_last_num} best accepted samples")
    elif not std_filter is None:
        ax.set_title(f"#Samples: {sum(res.ncall)}; {coverages[0]*100:.0f}% inside 2 sigma\n{filter_type}: {std_filter:.0e}")
    else:
        ax.set_title(f"#Samples: {sum(res.ncall)}; {coverages[0]*100:.0f}% inside 2 sigma\n")
    ax.set_xlabel(param_names[xdim])

    plt.colorbar(posterior_surface_scatter)

    if return_figure:
        return fig, ax
    if display_figure:
        plt.show()
    return None, None

## 2D

In [ ]:
def plot_3d_data(samples, xx, yy, return_figure=False, fig=None, ax=None, display_figure=True, title_add = "", shadow=True):
    """
    Similar to plot_3d_gp_samples, but color-codes each (xx, yy) point in 3D.
    'samples' can be a single 1D tensor or multiple samples in a 2D tensor.
    """
    if not (fig and ax):
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111, projection='3d')

    #if samples.ndim == 1:
    #    samples = samples.unsqueeze(0)

    #z_vals = samples.reshape(xx.shape)
    z_vals = samples
    ax.scatter(xx.numpy(), yy.numpy(), z_vals.numpy(),
                c=z_vals.numpy(), cmap='viridis', alpha=0.8)


    if shadow:
        # Plot shadows (projection on X-Y plane at z=0)
        ax.scatter(xx.numpy(), yy.numpy(), 
                np.ones_like(z_vals)*min(z_vals.numpy()), 
                c='gray', alpha=0.3, marker='o')



    ax.set_title(f'Data {title_add}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Output Value')
    if not return_figure and display_figure:
        plt.show()
    else:
        return fig, ax


def plot_3d_gp_samples(samples, xx, yy, return_figure=False, fig=None, ax=None, display_figure=True):
    """
    Visualize multiple samples drawn from a 2D-input (xx, yy) -> 1D-output GP in 3D.
    Each sample in 'samples' should be a 1D tensor that can be reshaped to match xx, yy.
    """
    if not (fig and ax):
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111, projection='3d')
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    if samples.ndim == 1:
        samples = samples.unsqueeze(0)
    for i, sample in enumerate(samples):
        z_vals = sample.reshape(xx.shape)
        ax.plot_surface(xx.numpy(), yy.numpy(), z_vals.numpy(), alpha=0.4)

    ax.set_title('GP Samples in 3D')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Output')
    if not return_figure and display_figure:
        plt.show()
    else:
        return fig, ax


def plot_3d_gp(model, likelihood, data=None, x_min=0.0, x_max=1.0, y_min=0.0, y_max=1.0,
                resolution=50, return_figure=False, fig=None, ax=None, 
                display_figure=True, loss_val=None, loss_type=None, shadow=False,
                title_add = ""):
    if not (fig and ax):
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111, projection='3d')


    if data is not None:
        if data.ndim == 1:
            data = data.unsqueeze(0)

        for sample in data:
            xx = sample[0]
            yy = sample[1]
            zz = sample[2]
            ax.scatter(xx.numpy(), yy.numpy(), zz.numpy(), color="red", alpha=0.8)

            if shadow:
                # Plot shadows (projection on X-Y plane at z=0)
                ax.scatter(xx.numpy(), yy.numpy(), 
                        min(data[:,2].numpy()), 
                        c='gray', alpha=0.3, marker='o')
    model.eval()
    likelihood.eval()

    x_vals = torch.linspace(x_min, x_max, resolution)
    y_vals = torch.linspace(y_min, y_max, resolution)
    xx, yy = torch.meshgrid(x_vals, y_vals)
    test_x = torch.stack([xx.reshape(-1), yy.reshape(-1)], dim=-1)

    with torch.no_grad():
        preds = likelihood(model(test_x))
        mean = preds.mean.reshape(resolution, resolution)
        lower, upper = preds.confidence_region()
        lower = lower.reshape(resolution, resolution)
        upper = upper.reshape(resolution, resolution)


    # Plot mean surface
    ax.plot_surface(xx.numpy(), yy.numpy(), mean.numpy(), cmap='viridis', alpha=0.8)

    # Plot lower and upper surfaces
    ax.plot_surface(xx.numpy(), yy.numpy(), lower.numpy(), color='gray', alpha=0.2)
    ax.plot_surface(xx.numpy(), yy.numpy(), upper.numpy(), color='gray', alpha=0.2)

    ax.set_title(f'2D GP in 3D {title_add}')
    if loss_val is not None:
        ax.set_title(f"{ax.title.get_text()}; {loss_type}: {loss_val}")
    ax.set_xlabel('X1')
    ax.set_ylabel('X2')
    ax.set_zlabel('Mean and Variance Range')

    if not return_figure and display_figure:
        plt.show()
    else:
        return fig, ax


# Definitions

In [ ]:
DatasetInfo = namedtuple('DatasetInfo', ['name', 'path', 'dimension', 'description'])
datasets_1d = {
    "alternating" : DatasetInfo('alternating', 'datasets/alternating.csv', 1, 'Alternates between high and low'),
    "broaden" : DatasetInfo('broaden', 'datasets/broaden.csv', 1, 'Data with increasing variance'),
    "large gap" : DatasetInfo('large gap', 'datasets/large_gap.csv', 1, 'Contains large gap between X values'),
    "linear outlier" : DatasetInfo('linear outlier', 'datasets/linear_outlier.csv', 1, 'Linear data with outlier in last entry'),
    "parabola" : DatasetInfo('parabola', 'datasets/parabola.csv', 1, 'Parabola'),
    "periodic linear" : DatasetInfo('periodic linear', 'datasets/periodic_linear.csv', 1, 'Periodic data that changes to becoming purely linear'),
    "periodic" : DatasetInfo('periodic', 'datasets/periodic.csv', 1, 'Periodic data'),
    "periodic_1D flattened" : DatasetInfo('periodic', 'datasets/periodic_1D flattened.csv', 1, 'Copy of the 2D dataset that effectively only happens on the X axis'),
    "v_lines" : DatasetInfo('v_lines', 'datasets/v_lines.csv', 1, 'Vertical lines'),
}



# named tuple or dictionary for the datasets, same as 1D case
DatasetInfo = namedtuple('DatasetInfo', ['name', 'path', 'dimension', 'description'])
datasets_2d = {
    "away": DatasetInfo('away', 'datasets/2D/away.csv', 2, 'Away from the center'),
    "bullseye": DatasetInfo('bullseye', 'datasets/2D/bullseye.csv', 2, 'Two circles forming a bullseye'),
    "circle": DatasetInfo('circle', 'datasets/2D/circle.csv', 2, 'Circle'),
    "dots": DatasetInfo('dots', 'datasets/2D/dots.csv', 2, 'An equidistant dot grid'),
    "dots favouring lower values": DatasetInfo('dots favouring lower values', 'datasets/2D/dots favouring lower values.csv', 2, 'An equidistant dot grid where the high point of Periodic_1D is alone and there are multiple duplicates of the low values'),
    "dots minimal": DatasetInfo('dots minimal', 'datasets/2D/dots minimal.csv', 2, 'An equidistant dot grid with 9 points'),
    "dots minimal no shift": DatasetInfo('dots minimal', 'datasets/2D/dots minimal no shift.csv', 2, 'An equidistant dot grid with 9 points'),
    "dots minimal shifted": DatasetInfo('dots minimal', 'datasets/2D/dots minimal shifted.csv', 2, 'An equidistant dot grid with 9 points, all shifted by 0.5 to each other'),
    "dots minimal shifted to zero": DatasetInfo('dots minimal', 'datasets/2D/dots minimal shifted to zero.csv', 2, 'Basically "dots minimal shifted", but such that zero is the centre.'),
    "dots minimal shifted around zero": DatasetInfo('dots minimal', 'datasets/2D/dots minimal shifted around zero.csv', 2, 'An equidistant dot grid with 9 points, centred around zero.'),
    "line": DatasetInfo('line', 'datasets/2D/line.csv', 2, 'One diagonal line across X-Y'),
    "line X": DatasetInfo('line', 'datasets/2D/line X.csv', 2, 'One diagonal line that strictly follows X'),
    "line Y": DatasetInfo('line', 'datasets/2D/line Y.csv', 2, 'One diagonal line that strictly follows Y'),
    "line 05X1Y": DatasetInfo('line', 'datasets/2D/line 05X1Y.csv', 2, 'One diagonal line across X-Y, where X is 0.5 slower than Y'),
    "h_lines": DatasetInfo('h_lines', 'datasets/2D/h_lines.csv', 2, 'Horizontal lines'),
    "high lines": DatasetInfo('high lines', 'datasets/2D/high_lines.csv', 2, 'Horizontal lines with a large gap between them'),
    "slant down": DatasetInfo('slant down', 'datasets/2D/slant_down.csv', 2, 'Straight lines slanting downwards'),
    "slant up": DatasetInfo('slant up', 'datasets/2D/slant_up.csv', 2, 'Straight lines slanting upwards'),
    "star": DatasetInfo('star', 'datasets/2D/star.csv', 2, 'Star shape'),
    "v_lines": DatasetInfo('v_lines', 'datasets/2D/v_lines.csv', 2, 'Vertical lines'),
    "wide lines": DatasetInfo('wide lines', 'datasets/2D/wide_lines.csv', 2, 'Vertical lines with a large gap between them'),
    "x_shape": DatasetInfo('x_shape', 'datasets/2D/x_shape.csv', 2, 'X shape'),
}

## GP definition

In [ ]:


class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel_text="RBF", weights=None):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()

        if kernel_text == "C*C*SE":
            self.covar_module = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()))
        elif kernel_text == "SE":
            self.covar_module = gpytorch.kernels.RBFKernel()
        elif kernel_text == "C*SE":
            self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        elif kernel_text == "LIN":
            self.covar_module = gpytorch.kernels.LinearKernel()
        elif kernel_text == "C*LIN":
            self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.LinearKernel())
        elif kernel_text == "LIN*SE":
            self.covar_module = gpytorch.kernels.LinearKernel() * gpytorch.kernels.RBFKernel()
        elif kernel_text == "LIN*PER":
            self.covar_module = gpytorch.kernels.LinearKernel() * gpytorch.kernels.PeriodicKernel()
        elif kernel_text == "SE+SE":
            self.covar_module =  gpytorch.kernels.RBFKernel() + gpytorch.kernels.RBFKernel()
        elif kernel_text == "RQ":
            self.covar_module = gpytorch.kernels.RQKernel()
        elif kernel_text == "PER":
            self.covar_module = gpytorch.kernels.PeriodicKernel()
        #elif kernel_text == "PER+SE":
        #    if weights is None:
        #        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel()) + gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        #    else:
        #        self.covar_module = weights[0]*gpytorch.kernels.PeriodicKernel() + weights[1]*gpytorch.kernels.RBFKernel()
        elif kernel_text == "PER*SE":
            self.covar_module = gpytorch.kernels.PeriodicKernel() * gpytorch.kernels.RBFKernel()
        #elif kernel_text == "PER*LIN":
        #    self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel()) * gpytorch.kernels.ScaleKernel(gpytorch.kernels.LinearKernel())
        elif kernel_text == "MAT32":
            self.covar_module = gpytorch.kernels.MaternKernel(nu=1.5)
        elif kernel_text == "MAT32+MAT52":
            self.covar_module =  gpytorch.kernels.MaternKernel(nu=1.5) + gpytorch.kernels.MaternKernel()
        elif kernel_text == "MAT32*PER":
            self.covar_module =  gpytorch.kernels.MaternKernel(nu=1.5) * gpytorch.kernels.PeriodicKernel()
        elif kernel_text == "MAT32+PER":
            self.covar_module =  gpytorch.kernels.MaternKernel(nu=1.5) + gpytorch.kernels.PeriodicKernel()
        elif kernel_text == "MAT32*SE":
            self.covar_module =  gpytorch.kernels.MaternKernel(nu=1.5) * gpytorch.kernels.RBFKernel()
        elif kernel_text == "MAT32+SE":
            self.covar_module =  gpytorch.kernels.MaternKernel(nu=1.5) + gpytorch.kernels.RBFKernel()
        elif kernel_text == "MAT52":
            self.covar_module = gpytorch.kernels.MaternKernel()
        elif kernel_text == "MAT52*PER":
            self.covar_module =  gpytorch.kernels.MaternKernel() * gpytorch.kernels.PeriodicKernel()
        elif kernel_text == "MAT52+SE":
            self.covar_module =  gpytorch.kernels.MaternKernel() + gpytorch.kernels.RBFKernel()
        elif kernel_text == "SE*SE":
            self.covar_module =  gpytorch.kernels.RBFKernel() * gpytorch.kernels.RBFKernel()
        elif kernel_text == "(SE+RQ)*PER":
            self.covar_module =  (gpytorch.kernels.RBFKernel() + gpytorch.kernels.RQKernel()) * gpytorch.kernels.PeriodicKernel()
        elif kernel_text == "SE+SE+SE":
            self.covar_module =  gpytorch.kernels.RBFKernel() + gpytorch.kernels.RBFKernel() + gpytorch.kernels.RBFKernel()
        elif kernel_text == "MAT32+(MAT52*PER)":
            self.covar_module =  gpytorch.kernels.MaternKernel(nu=1.5) + (gpytorch.kernels.MaternKernel() * gpytorch.kernels.PeriodicKernel())

        #elif kernel_text == "RQ*PER":
        #    self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RQKernel()) * gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel())
        #elif kernel_text == "RQ*MAT32":
        #    self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RQKernel()) * gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=1.5))
        #elif kernel_text == "RQ*SE":
        #    self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RQKernel()) * gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)



class ExactMIGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel_text="SE", weights=None):
        super(ExactMIGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        if kernel_text == "[SE; SE]":
            self.covar_module = gpytorch.kernels.AdditiveStructureKernel(gpytorch.kernels.RBFKernel(active_dims=0) + gpytorch.kernels.RBFKernel(active_dims=1), num_dims=2)
        elif kernel_text == "[SE+SE; 1]2":
            self.covar_module = gpytorch.kernels.RBFKernel(active_dims=0) + gpytorch.kernels.RBFKernel(active_dims=0)
        elif kernel_text == "[SE+SE; 1]":
            self.covar_module = gpytorch.kernels.AdditiveStructureKernel(gpytorch.kernels.RBFKernel(active_dims=0) + gpytorch.kernels.RBFKernel(active_dims=0), num_dims=2)
        elif kernel_text == "[SE;* SE]":
            #TODO
            self.covar_module = gpytorch.kernels.AdditiveStructureKernel(gpytorch.kernels.RBFKernel(active_dims=0) + gpytorch.kernels.RBFKernel(active_dims=1), num_dims=2)
        elif kernel_text == "[SE; LIN]":
            self.covar_module = gpytorch.kernels.AdditiveStructureKernel(gpytorch.kernels.RBFKernel(active_dims=0) + gpytorch.kernels.LinearKernel(active_dims=1), num_dims=2)
        elif kernel_text == "[LIN; SE]":
            self.covar_module = gpytorch.kernels.AdditiveStructureKernel(gpytorch.kernels.LinearKernel(active_dims=0) + gpytorch.kernels.RBFKernel(active_dims=1), num_dims=2)
        elif kernel_text == "[LIN; LIN]":
            self.covar_module = gpytorch.kernels.AdditiveStructureKernel(gpytorch.kernels.LinearKernel(active_dims=0) + gpytorch.kernels.LinearKernel(active_dims=1), num_dims=2)
        elif kernel_text == "[LIN;* LIN]":
            self.covar_module = gpytorch.kernels.LinearKernel(active_dims=0) * gpytorch.kernels.LinearKernel(active_dims=1)
        elif kernel_text == "[SE;* 1]":
            self.covar_module = gpytorch.kernels.RBFKernel(active_dims=0)
        elif kernel_text == "[PER;* 1]":
            self.covar_module = gpytorch.kernels.PeriodicKernel(active_dims=0)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


# Various 1D and 2D datasets 

## 1D data

In [ ]:
possible_kenels = [
    "SE",
    "C*SE",
    "C*C*SE",
    "LIN",
    "C*LIN",
    "LIN*SE",
    "LIN*PER",
    "SE+SE",
    "RQ",
    "PER",
    "PER*SE",
    "MAT32",
    "MAT32+MAT52",
    "MAT32*PER",
    "MAT32+PER",
    "MAT32*SE",
    "MAT32+SE",
    "MAT52",
    "MAT52*PER",
    "MAT52+SE",
    "SE*SE",
    "(SE+RQ)*PER",
    "SE+SE+SE",
    "MAT32+(MAT52*PER)"]

possible_datasets = [
   "alternating",
   "broaden",
   "large gap",
   "linear outlier",
   "parabola",
   "periodic linear",
   "periodic",
    "periodic_1D flattened",
   "v_lines"
]

kernel_name = "SE+SE"
dataset_name = "periodic_1D flattened"
dataset_addendum = ""#"51 points"
logarithmic_reinit=True
data_normalization = True
data_norm_y = False 
levels = [1e+4, 1e+5]
uninformed = True

log_path = f"logs/{'x-normalized' if data_normalization else ''}_{dataset_name}_{dataset_addendum}/{kernel_name}"
if not os.path.exists(log_path):
    os.makedirs(log_path)

In [ ]:
# Load the dataset
if dataset_name in datasets_1d:
    df = pd.read_csv(datasets_1d[dataset_name].path, header=None)
    train_x = torch.tensor(df[0], dtype=torch.float32)
    train_y = torch.tensor(df[1], dtype=torch.float32)

elif dataset_name == "linear":
    train_x = torch.linspace(0, int(dataset_addendum.split(" ")[0])-1, int(dataset_addendum.split(" ")[0]))
    train_y = train_x.clone()
    #train_y[-1] = 0 


# Z score normalization
if data_normalization:
    if not dataset_name == "periodic_1D flattened":
        train_x = (train_x - train_x.mean()) / train_x.std()
    if data_norm_y:
        train_y = (train_y - train_y.mean()) / train_y.std()

#train_x = torch.tensor([0.0, 1.0])
#train_y = torch.tensor([0.0, 1.0])


fig, ax = plot_data(train_x, train_y, title_add=dataset_name, return_figure=True)
fig.savefig(f"{log_path}/data.png", bbox_inches='tight')
#fig.savefig(f"{log_path}/data.pgf", bbox_inches='tight')

In [ ]:
one_dim_data = (train_x, train_y)

In [ ]:
print(min(train_y))
print(max(train_y))
print(min(train_x))
print(max(train_x))

In [ ]:
len(train_x)

In [ ]:
## Define the GP model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood, kernel_text=kernel_name)
likelihood_MAP = gpytorch.likelihoods.GaussianLikelihood()
model_MAP = ExactGPModel(train_x, train_y, likelihood_MAP, kernel_text=kernel_name)

In [ ]:
model

In [ ]:
param_names = [l[0] for l in list(model.named_parameters())]

In [ ]:
random_restarts = 4

## Train the GPs
model.train()
likelihood.train()
neg_scaled_mll_loss, model, likelihood, mll_train_log = optimize_hyperparameters(model, likelihood, X=train_x, Y=train_y, MAP=False, uninformed=uninformed, random_restarts=random_restarts)
mll_opt_params = [p.item() for p in model.parameters() if p.requires_grad]

In [ ]:
# Give me the index of the best training run
best_mll_training_run = np.argmin([min(run) for run in mll_train_log.neg_loss])
best_mll_training_run

In [ ]:
# Plot the optimization path of the trainings, each in an individual plot, given the parameter of interest
# The parameter progression is plotted as arrows pointing to the next parameter position
# The arrows are colored according to the lateness of their progression
# The colorbar shows the progression of the parameter



f, axs = plt.subplots(1, (random_restarts), figsize=(4*(random_restarts), 4), sharey=True, layout="constrained")
if random_restarts == 1:
    axs = [axs]

xdim = 0
ydim = 1
# extract all the parameters
## Likelihood values

#mll_train_log.neg_loss
## Parameters
#mll_train_log.x
parameter_paths = [[log for log in mll_train_log.x[i]] for i in range(len(mll_train_log.x))]
parameter_values = [[log for log in mll_train_log.neg_loss[i]] for i in range(len(mll_train_log.neg_loss))]

for i, ax in enumerate(axs):
    # train log consists of a list of tuples, where the first element is the parameter vector and the second element is the loss
    plot_parameter_progression(parameter_paths[i], losses=parameter_values[i], xlabel=param_names[xdim], ylabel=param_names[ydim], xdim=xdim, ydim=ydim, fig=f, ax=ax, display_figure=False, title_add=f"restart {i}")

In [ ]:
model_MAP.train()
likelihood_MAP.train()
neg_scaled_map_loss, model_MAP, likelihood_MAP, map_train_log = optimize_hyperparameters(model_MAP, likelihood_MAP, X=train_x, Y=train_y, MAP=True, uninformed=uninformed, random_restarts=random_restarts)
map_opt_params = [p.item() for p in model_MAP.parameters() if p.requires_grad]

In [ ]:
print([torch.nn.functional.softplus(p) for p in model_MAP.parameters() if p.requires_grad]) 

In [ ]:
# Give me the index of the best training run
best_map_training_run = np.argmin([min(run) for run in map_train_log.neg_loss])
best_map_training_run

### Exploring the PyGRANSO Hessian

In [ ]:
map_train_log.hessians[best_map_training_run]

In [ ]:
torch.linalg.det(map_train_log.hessians[best_map_training_run][-1])

In [ ]:
mll_train_log.hessians[best_mll_training_run][-1]

In [ ]:


f, axs = plt.subplots(1, (random_restarts), figsize=(4*(random_restarts), 4), sharey=True, layout="constrained")
if random_restarts == 1:
    axs = [axs]

# extract all the parameters
parameter_paths = [[log for log in map_train_log.x[i]] for i in range(len(map_train_log.x))]
parameter_values = [[log for log in map_train_log.neg_loss[i]] for i in range(len(map_train_log.neg_loss))]

xdim = 0
ydim = 1

for i, ax in enumerate(axs):
    plot_parameter_progression(parameter_paths[i], losses=parameter_values[i], xlabel=param_names[xdim], ylabel=param_names[ydim], xdim=xdim, ydim=ydim, fig=f, ax=ax, display_figure=False, title_add=f"restart {i}")

In [ ]:
one_dim_train_log = map_train_log 

In [ ]:
# One dimensional case:
#Final best parameters: [('likelihood.noise_covar.raw_noise', Parameter containing:
#tensor([-11.6140], requires_grad=True)), ('covar_module.kernels.0.raw_lengthscale', Parameter containing:
#tensor([[-0.3335]], requires_grad=True)), ('covar_module.kernels.1.raw_lengthscale', Parameter containing:
#tensor([[-0.3335]], requires_grad=True))] w. loss: 0.6454170942306519 (smaller=better)
# Two dimensional case:
#Final best parameters: [('likelihood.noise_covar.raw_noise', Parameter containing:
#tensor([-11.7261], requires_grad=True)), ('covar_module.base_kernel.kernels.0.raw_lengthscale', Parameter containing:
#tensor([[-0.3326]], requires_grad=True)), ('covar_module.base_kernel.kernels.1.raw_lengthscale', Parameter containing:
#tensor([[-0.3326]], requires_grad=True))] w. loss: 0.6454619765281677 (smaller=better)

#fixed_reinit(model_MAP, torch.tensor([-11.7261, -0.3326, -0.3326]))

In [ ]:
one_dim_mean_vec = model_MAP(train_x).mean
one_dim_cov_matr = model_MAP(train_x).covariance_matrix


In [ ]:
model.eval()
likelihood.eval()
model_MAP.eval()
likelihood_MAP.eval()

In [ ]:
one_dim_post_mean_vec = model_MAP(train_x).mean
one_dim_post_cov_matr = model_MAP(train_x).covariance_matrix

In [ ]:
model.eval()
likelihood.eval()
model_MAP.eval()
likelihood_MAP.eval()
fig, axs = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
ax = axs[0]
ax_MAP = axs[1]
plot_model(model, likelihood, train_x, train_y, return_figure=False, figure=fig, ax=ax, loss_val=neg_scaled_mll_loss.item(), loss_type="mll", display_figure=False)
plot_model(model_MAP, likelihood_MAP, train_x, train_y, return_figure=False, figure=fig, ax=ax_MAP, loss_val=neg_scaled_map_loss.item(), loss_type="map")
fig.savefig(f"{log_path}/posterior.png", bbox_inches='tight')
#fig.savefig(f"{log_path}/posterior.pgf", bbox_inches='tight')
model.train()
likelihood.train()
model_MAP.train()
likelihood_MAP.train()

In [ ]:
neg_scaled_map_loss

In [ ]:
model.train()
likelihood.train()
model_MAP.train()
likelihood_MAP.train()

In [ ]:
pos_unscaled_MLL = -neg_scaled_mll_loss*len(*model.train_inputs)
pos_unscaled_MAP = -neg_scaled_map_loss*len(*model.train_inputs)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

bool_use_finite_difference_hessian = True

## Calculate the metrics
AIC_val, AIC_logs = AIC(pos_unscaled_MLL, num_params)
BIC_val, BIC_logs = BIC(pos_unscaled_MLL, num_params, torch.tensor(len(train_x)))
AIC_MAP_val, AIC_MAP_logs = AIC(pos_unscaled_MAP, num_params)
BIC_MAP_val, BIC_MAP_logs = BIC(pos_unscaled_MAP, num_params, torch.tensor(len(train_x)))
Lap0_val, Lap0_logs =     Laplace(model_MAP, pos_unscaled_MAP.clone(), uninformed=uninformed, use_finite_difference_hessian=bool_use_finite_difference_hessian, param_punish_term=0)
LapAIC_val, LapAIC_logs = Laplace(model_MAP, pos_unscaled_MAP.clone(), uninformed=uninformed, use_finite_difference_hessian=bool_use_finite_difference_hessian, param_punish_term=-1)
LapBIC_val, LapBIC_logs = Laplace(model_MAP, pos_unscaled_MAP.clone(), uninformed=uninformed, use_finite_difference_hessian=bool_use_finite_difference_hessian, param_punish_term="BIC")

In [ ]:
Lap0_logs["original symmetrized Hessian"]

In [ ]:
h_step = 5e-1
h_i_step = 0.5*h_step 
h_j_step = 0.5*h_step

hessian_finite_differences_neg_unscaled_map = np.zeros((num_params, num_params))
for h_step in [1e-1, 5e-1, 1e-2, 5e-2, 1e-3, 5e-3]:
    print(h_step)
    h_i_step = 0.5*h_step 
    h_j_step = 0.5*h_step
    for i, j in itertools.product(range(num_params), range(num_params)):
        h_i_vec = np.zeros(num_params)
        h_j_vec = np.zeros(num_params)
        h_i_vec[i] = 1.0
        h_j_vec[j] = 1.0
        hessian_finite_differences_neg_unscaled_map[i][j] = finite_difference_second_derivative_GP_neg_unscaled_map(model_MAP, likelihood_MAP, train_x, train_y, uninformed=uninformed, h_i_step=h_i_step, h_j_step=h_j_step, h_i_vec=h_i_vec, h_j_vec=h_j_vec)
    print(hessian_finite_differences_neg_unscaled_map)

In [ ]:
# calculate the hessian of the unscaled MLL
params_list = [p for p in model.parameters()]
neg_unscaled_map = -pos_unscaled_MAP
try:
    jacobian_neg_unscaled_map = torch.autograd.grad(neg_unscaled_map, params_list, retain_graph=True, create_graph=True, allow_unused=True)
except Exception as E:
    print(E)
    import pdb
    pdb.set_trace()
    print(f"E:{E}")
hessian_neg_unscaled_map = []
# Calcuate -\nabla\nabla log(f(\theta)) (i.e. Hessian of negative log marginal likelihood)
for i in range(len(jacobian_neg_unscaled_mll)):
    hessian_neg_unscaled_map.append(torch.autograd.grad(jacobian_neg_unscaled_map[i], params_list, retain_graph=True, allow_unused=True)) 

In [ ]:
hessian_neg_unscaled_map

In [ ]:
Lap0_logs

In [ ]:
LapAIC_logs

In [ ]:
LapBIC_logs

In [ ]:
model_evidences = list()
model_evidence_logs = list()
uninformed = True

In [ ]:
maxcall = 1e+1
if maxcall in levels:
    print(maxcall)
    nested_pos_model_evidence_e1, Nested_logs_e1 = Nested(model, store_full=True, pickle_directory=os.path.join(log_path, "Nested_log"), maxcall=maxcall)
    model_evidences.append(nested_pos_model_evidence_e1)
    model_evidence_logs.append(Nested_logs_e1)

In [ ]:
maxcall = 1e+2
if maxcall in levels:
    print(maxcall)
    nested_pos_model_evidence_e2, Nested_logs_e2 = Nested(model, store_full=True, pickle_directory=os.path.join(log_path, "Nested_log"), maxcall=maxcall)
    model_evidences.append(nested_pos_model_evidence_e2)
    model_evidence_logs.append(Nested_logs_e2)

In [ ]:
maxcall = 1e+3
if maxcall in levels:
    print(maxcall)
    nested_pos_model_evidence_e3, Nested_logs_e3 = Nested(model, store_full=True, pickle_directory=os.path.join(log_path, "Nested_log"), maxcall=maxcall, uninformed=uninformed)
    model_evidences.append(nested_pos_model_evidence_e3)
    model_evidence_logs.append(Nested_logs_e3)

In [ ]:
maxcall = 1e+4
if maxcall in levels:
    print(maxcall)
    nested_pos_model_evidence_e4, Nested_logs_e4 = Nested(model, store_full=True, pickle_directory=os.path.join(log_path, "Nested_log"), maxcall=maxcall, uninformed=uninformed)
    model_evidences.append(nested_pos_model_evidence_e4)
    model_evidence_logs.append(Nested_logs_e4)

In [ ]:
maxcall = 1e+5
if maxcall in levels:
    print(maxcall)
    nested_pos_model_evidence_e5, Nested_logs_e5 = Nested(model, store_full=True, pickle_directory=os.path.join(log_path, "Nested_log"), maxcall=maxcall, uninformed=uninformed)
    model_evidences.append(nested_pos_model_evidence_e5)
    model_evidence_logs.append(Nested_logs_e5)

In [ ]:
for mod_log in model_evidence_logs:
    with open(f"{mod_log['res file']}", "rb") as f:
        res = dill.load(f)
    print(f"Num calls: {sum(res.ncall)}")
    print(f"logz Error: {res["logzerr"][-1]}")

In [ ]:

plot_mll_opt = True
std_filter = None # float as 1e-3 or None
show_last_num = 0.5 # integer, float or None
filter_type = "none" # ("mean"), "max", "none"
xdim = 1
ydim = 2

param_names = [l[0] for l in list(model.named_parameters())]

f, axs = plt.subplots(1, len(levels), figsize=(4*len(levels), 4), sharey=True, layout="constrained")
if len(levels) == 1:
    axs = [axs]
for ax, level, model_evidence_log in zip(axs, levels, model_evidence_logs):
    if ax == axs[-1]:
        nested_sampling_plot(model, model_evidence_log, xdim = xdim, ydim = ydim, filter_type=filter_type, std_filter=std_filter, show_last_num=show_last_num, return_figure=False, title_add="", fig=f, ax=ax, display_figure=False, plot_mll_opt=plot_mll_opt, mll_opt_params=mll_opt_params, plot_lap=True, Lap0_logs=Lap0_logs, LapAIC_logs=LapAIC_logs, LapBIC_logs=LapBIC_logs, lap_colors = ["r", "pink", "white"], Lap_hess=Lap0_logs["original symmetrized Hessian"])
    else:
        nested_sampling_plot(model, model_evidence_log, xdim = xdim, ydim = ydim, filter_type=filter_type, std_filter=std_filter, show_last_num=show_last_num, return_figure=False, title_add="", fig=f, ax=ax, display_figure=False, plot_mll_opt=plot_mll_opt, mll_opt_params=mll_opt_params, plot_lap=True, Lap0_logs=Lap0_logs, LapAIC_logs=LapAIC_logs, LapBIC_logs=LapBIC_logs, lap_colors = ["r", "pink", "white"], Lap_hess=Lap0_logs["original symmetrized Hessian"])
    if ax == axs[0] or len(levels)==1:
        ax.set_ylabel(param_names[ydim])

In [ ]:

map_opt_params = [p.item() for p in model_MAP.parameters() if p.requires_grad]
plot_map_opt = False
std_filter = None # float as 1e-3 or None
show_last_num = 0.4 # integer, float or None
filter_type = "none" # ("mean"), "max", "none"
xdim = 0
ydim = 1

param_names = [l[0] for l in list(model.named_parameters())]

f, axs = plt.subplots(1, len(levels), figsize=(4*len(levels), 4), sharey=True, layout="constrained")
if len(levels) == 1:
    axs = [axs]
for ax, level, model_evidence_log in zip(axs, levels, model_evidence_logs):
    if ax == axs[-1]:
        posterior_surface_plot(model, model_evidence_log, xdim = xdim, ydim = ydim, filter_type=filter_type, std_filter=std_filter, show_last_num=show_last_num, return_figure=False, title_add="", fig=f, ax=ax, display_figure=False, plot_mll_opt=plot_map_opt, mll_opt_params=map_opt_params, plot_lap=True, Lap0_logs=Lap0_logs, LapAIC_logs=LapAIC_logs, LapBIC_logs=LapBIC_logs, lap_colors = ["r", "pink", "white"], uninformed=uninformed, Lap_hess=Lap0_logs["original symmetrized Hessian"])
    else:
        posterior_surface_plot(model, model_evidence_log, xdim = xdim, ydim = ydim, filter_type=filter_type, std_filter=std_filter, show_last_num=show_last_num, return_figure=False, title_add="", fig=f, ax=ax, display_figure=False, plot_mll_opt=plot_map_opt, mll_opt_params=map_opt_params, plot_lap=True, Lap0_logs=Lap0_logs, LapAIC_logs=LapAIC_logs, LapBIC_logs=LapBIC_logs, lap_colors = ["r", "pink", "white"], uninformed=uninformed, Lap_hess=Lap0_logs["original symmetrized Hessian"])
    if ax == axs[0] or len(levels)==1:
        ax.set_ylabel(param_names[ydim])

In [ ]:
# Find the best value and the corresponding hyperparameters
best_idx = np.argmax(res.logl)
best_hyperparameters = res.samples[best_idx]

In [ ]:
# Optimal GP according to Nested sampling
fixed_reinit(model, torch.tensor(best_hyperparameters))
model.eval()
likelihood.eval()
# mll of best parameterization according to Nested sampling:
mll_fkt = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
unscaled_pos_nested_top_mll = (mll_fkt(model(train_x), train_y))*len(train_x)
unscaled_pos_nested_top_map = unscaled_pos_nested_top_mll+log_normalized_prior(model)*len(train_x)
fig, ax = plot_model(model, likelihood, train_x, train_y, return_figure=True, loss_val=res.logl[best_idx], loss_type="Nested logl")
ax.title.set_text(f"{ax.title.get_text()} ; MLL: {unscaled_pos_nested_top_mll.item():.2f}; MAP: {unscaled_pos_nested_top_map.item():.2f}\n MLL (u) {-unscaled_pos_nested_top_mll/len(train_x):.2f}; MAP (u) {-unscaled_pos_nested_top_map/len(train_x):.2f}")

In [ ]:
# Collect metric names and values into a list of tuples
metrics_data = [
    ("MLL", f"{pos_unscaled_MLL.item():.3f}"),
    ("MLL (loss)", f"{neg_scaled_mll_loss.item():.3f}"),
    ("AIC", f"{AIC_val.item():.3f}"),
    ("AIC (s)", f"{(AIC_val.item()*(-0.5)):.3f}"),
    ("BIC", f"{BIC_val.item():.3f}"),
    ("BIC (s)", f"{(BIC_val.item()*(-0.5)):.3f}"),
    

]
labels = [m[0] for m in metrics_data]
values = [m[1] for m in metrics_data]
# Print in transposed form
print("|        | " + " | ".join(labels) + " |")
print("|--------|" + "|".join(["-----------" for _ in labels]) + "|")
print("| Value  | " + " | ".join(values) + " |")
print()

metrics_data = [("MAP", f"{pos_unscaled_MAP.item():.3f}"),
    ("MAP (loss)", f"{neg_scaled_map_loss.item():.3f}"),
    ("AIC_M", f"{AIC_MAP_val.item():.3f}"),
    ("AIC_M (s)", f"{(AIC_MAP_val.item()*(-0.5)):.3f}"),
    ("BIC_M", f"{BIC_MAP_val.item():.3f}"),
    ("BIC_M (s)", f"{(BIC_MAP_val.item()*(-0.5)):.3f}"),
]

labels = [m[0] for m in metrics_data]
values = [m[1] for m in metrics_data]
# Print in transposed form
print("|        | " + " | ".join(labels) + " |")
print("|--------|" + "|".join(["-----------" for _ in labels]) + "|")
print("| Value  | " + " | ".join(values) + " |")
print()


metrics_data=[    
    ("Lap", f"{Lap0_logs["laplace without replacement"].item():.3f}"),
    ("Lap0", f"{Lap0_val.item():.3f}"),
    ("LapAIC", f"{LapAIC_val.item():.3f}"),
    ("LapBIC", f"{LapBIC_val.item():.3f}"),
    ]


labels = [m[0] for m in metrics_data]
values = [m[1] for m in metrics_data]
# Print in transposed form
print("|        | " + " | ".join(labels) + " |")
print("|--------|" + "|".join(["-----------" for _ in labels]) + "|")
print("| Value  | " + " | ".join(values) + " |")
print()
metrics_data = [    
    #("Nested (1)", f"{Nested_val_e1:.3f}"),
    #("Nested (3)", f"{Nested_val_e3:.3f}"),
    ("Nested (4)", f"{nested_pos_model_evidence_e4:.3f}"),
    ("Nested (5)", f"{nested_pos_model_evidence_e5:.3f}")
    ]
# Transpose the table: one row for the label, another for the value
labels = [m[0] for m in metrics_data]
values = [m[1] for m in metrics_data]

# Print in transposed form
print("|        | " + " | ".join(labels) + " |")
print("|--------|" + "|".join(["-----------" for _ in labels]) + "|")
print("| Value  | " + " | ".join(values) + " |")

In [ ]:

fixed_reinit(model_MAP, torch.tensor(best_hyperparameters))
mll_fkt = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood_MAP, model_MAP)
neg_scaled_map_loss = -mll_fkt(model_MAP(train_x), train_y)-log_normalized_prior(model_MAP)
pos_unscaled_MAP = -neg_scaled_map_loss*len(*model.train_inputs)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
Nested_Lap0_val, Nested_Lap0_logs = Laplace(model_MAP, pos_unscaled_MAP, param_punish_term=0)

print(f"Lap/nested = {Lap0_logs["laplace without replacement"].item()/nested_pos_model_evidence_e5:.3f}"),
print(f"Lap0/nested = {Lap0_val/nested_pos_model_evidence_e5:.3f}")
print(f"AIC (s)/nested = {AIC_val*(-0.5)/nested_pos_model_evidence_e5:.3f}") 
print(f"nested Lap0/nested = {Nested_Lap0_val/nested_pos_model_evidence_e5:.3f}\n")
print("```python")
print(Nested_Lap0_val)

print(Nested_Lap0_logs)
print("```")

---

## 2D data

In [ ]:

possible_datasets = {
    "periodic_1D" : periodic_1D,
    "periodic_2D" : periodic_2D,
    "parabola_1D" : parabola_1D,
    "parabola_2D" : parabola_2D,
    "product" : product,
    "periodic_sum": periodic_sum,
    "periodic_sincos": periodic_sincos,
    "linear_1D": linear_1D,
    "linear_2D": linear_2D,
}

possible_X_patterns = [
    "away",
    "bullseye",
    "circle",
    "dots",
    "dots minimal",
    "dots minimal no shift",
    "dots minimal shifted",
    "dots minimal shifted around zero",
    "dots minimal shifted to zero",
    "dots favouring lower values",
    "line",
    "line X",
    "line 05X1Y",
    "h_lines",
    "high lines",
    "slant down",
    "slant up",
    "star",
    "v_lines",
    "wide lines",
    "x_shape",
]

possible_kenels = [
        "[SE; SE]", #0
        "[SE; LIN]",
        "[LIN; SE]",
        "[LIN; LIN]", #3
        "[LIN;* LIN]",
        "[SE;* 1]", #5
        "[PER;* 1]", #6
        "[SE+SE; 1]", #7 #BOO. Fuck AdditiveStructureKernels. All my Homies hate AdditiveStructureKernels
        "[SE+SE; 1]2" #8
    ]
kernel_name = possible_kenels[1]
dataset_name = "periodic_1D"
pattern = "line X"

dataset_addendum = ""#"51 points"
logarithmic_reinit=True
data_normalization = True
data_norm_y = True 
X1_noise = 0.0
X2_noise = 0.0
levels = [1e+4, 1e+5]
uninformed = True


log_path = f"logs/2D/{'x-normalized' if data_normalization else ''}_{dataset_name}-{pattern}_{dataset_addendum}/{kernel_name}"
if not os.path.exists(log_path):
    os.makedirs(log_path)

In [ ]:

# Load the dataset
if pattern in datasets_2d:
    df = pd.read_csv(datasets_2d[pattern].path, header=None)
    x1 = torch.tensor(df[0], dtype=torch.float32) + X1_noise*torch.randn_like(torch.tensor(df[0], dtype=torch.float32))
    x2 = torch.tensor(df[1], dtype=torch.float32) + X2_noise*torch.randn_like(torch.tensor(df[1], dtype=torch.float32))
    train_x = torch.stack([x1, x2], dim=-1)
    print(train_x)

# Generate the z axis
train_y = possible_datasets[dataset_name](train_x)
# Z score normalization
if data_normalization:
    print(f"X-scale: {train_x.std()}")
    print(f"Y-scale: {train_y.std()}")
    train_x = (train_x - train_x.mean()) / train_x.std()
    if data_norm_y:
        train_y = (train_y - train_y.mean()) / train_y.std()

    print(f"New distribution\n X: {train_x.mean()} {train_x.std()}\n Y: {train_y.mean()} {train_y.std()}")

#train_x = torch.tensor([[0.0, 0.0], [1.0, 0.0]])
#train_y = torch.tensor([0.0, 1.0])

fig, ax = plot_3d_data(train_y, train_x[:,0], train_x[:,1], title_add=f"{dataset_name}-{pattern}", return_figure=True)
fig.savefig(f"{log_path}/data.png", bbox_inches='tight')
#fig.savefig(f"{log_path}/data.pgf", bbox_inches='tight')

In [ ]:
two_dim_data = (train_x, train_y)

In [ ]:
len(train_x)

In [ ]:
## Define the GP model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactMIGPModel(train_x, train_y, likelihood, kernel_text=kernel_name)
likelihood_MAP = gpytorch.likelihoods.GaussianLikelihood()
model_MAP = ExactMIGPModel(train_x, train_y, likelihood_MAP, kernel_text=kernel_name)

In [ ]:
model

In [ ]:
param_names = [l[0] for l in list(model.named_parameters())]

In [ ]:
random_restarts = 4
## Train the GPs
model.train()
likelihood.train()
neg_scaled_mll_loss, model, likelihood, mll_train_log = optimize_hyperparameters(model, likelihood, X=train_x, Y=train_y, MAP=False, uninformed=uninformed, random_restarts=random_restarts, logarithmic_reinit=logarithmic_reinit)
mll_opt_params = [p.item() for p in model.parameters() if p.requires_grad]

In [ ]:
# Plot the optimization path of the trainings, each in an individual plot, given the parameter of interest
# The parameter progression is plotted as arrows pointing to the next parameter position
# The arrows are colored according to the lateness of their progression
# The colorbar shows the progression of the parameter



f, axs = plt.subplots(1, (random_restarts), figsize=(4*(random_restarts), 4), sharey=True, layout="constrained")
if random_restarts == 1:
    axs = [axs]

xdim = 0
ydim = 1
# extract all the parameters
parameter_paths = [[log for log in mll_train_log.x[i]] for i in range(len(mll_train_log.x))]
parameter_values = [[log for log in mll_train_log.neg_loss[i]] for i in range(len(mll_train_log.neg_loss))]

for i, ax in enumerate(axs):
    if len(parameter_values[i]) == 1:
        continue
    # train log consists of a list of tuples, where the first element is the parameter vector and the second element is the loss
    plot_parameter_progression(parameter_paths[i], losses=parameter_values[i], xlabel=param_names[xdim], ylabel=param_names[ydim], xdim=xdim, ydim=ydim, fig=f, ax=ax, display_figure=False, title_add=f"restart {i}")

In [ ]:
model_MAP.train()
likelihood_MAP.train()
neg_scaled_map_loss, model_MAP, likelihood_MAP, map_train_log = optimize_hyperparameters(model_MAP, likelihood_MAP, X=train_x, Y=train_y, MAP=True, uninformed=uninformed, random_restarts=random_restarts, logarithmic_reinit=logarithmic_reinit)
map_opt_params = [p.item() for p in model_MAP.parameters() if p.requires_grad]

In [ ]:
# Give me the index of the best training run
best_map_training_run = np.argmin([min(run) for run in map_train_log.neg_loss])
best_map_training_run

In [ ]:
map_train_log.hessians[best_map_training_run]

In [ ]:
print(list(model_MAP.parameters()))
neg_scaled_map_loss

In [ ]:
print([torch.nn.functional.softplus(p) for p in model_MAP.parameters() if p.requires_grad]) 

In [ ]:


f, axs = plt.subplots(1, (random_restarts), figsize=(4*(random_restarts), 4), sharey=True, layout="constrained")
if random_restarts == 1:
    axs = [axs]

# extract all the parameters
parameter_paths = [[log for log in map_train_log.x[i]] for i in range(len(map_train_log.x))]
parameter_values = [[log for log in map_train_log.neg_loss[i]] for i in range(len(map_train_log.neg_loss))]

xdim = 0
ydim = 1

for i, ax in enumerate(axs):
    if len(parameter_values[i]) == 1:
        continue
    plot_parameter_progression(parameter_paths[i], losses=parameter_values[i], xlabel=param_names[xdim], ylabel=param_names[ydim], xdim=xdim, ydim=ydim, fig=f, ax=ax, display_figure=False, title_add=f"restart {i}")

In [ ]:
two_dim_train_log = map_train_log 

In [ ]:
two_dim_mean_vec = model_MAP(train_x).mean
two_dim_cov_matr = model_MAP(train_x).covariance_matrix

In [ ]:
model.eval()
likelihood.eval()
model_MAP.eval()
likelihood_MAP.eval()

In [ ]:
two_dim_post_mean_vec = model_MAP(train_x).mean
two_dim_post_cov_matr = model_MAP(train_x).covariance_matrix

In [ ]:
model.eval()
likelihood.eval()
model_MAP.eval()
likelihood_MAP.eval()
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(1, 2, 1, projection='3d')
ax_MAP = fig.add_subplot(1, 2, 2, projection='3d')
#ax = axs[0]
#ax_MAP = axs[1]
x_min=min(train_x[:,0])
x_max=max(train_x[:,0])
y_min=min(train_x[:,1])
y_max=max(train_x[:,1])

title_add = ""
if y_min == y_max:
    y_min = y_min-0.1
    y_max = y_max+0.1
    title_add += " Synthetic y stretch"
if x_min == x_max:
    x_min = x_min-0.1
    x_max = x_max+0.1
    title_add += " Synthetic x stretch"

plot_3d_gp(model, likelihood, data=torch.stack([train_x[:,0], train_x[:,1], train_y], -1), x_min=x_min, x_max=x_max, y_min=y_min, y_max=y_max, return_figure=False, fig=fig, ax=ax, loss_val=np.round(neg_scaled_mll_loss.item(), 2), loss_type="mll", display_figure=False, shadow=False, title_add=title_add)
plot_3d_gp(model_MAP, likelihood_MAP, data=torch.stack([train_x[:,0], train_x[:,1], train_y], -1), x_min=x_min, x_max=x_max, y_min=y_min, y_max=y_max,return_figure=False, fig=fig, ax=ax_MAP, loss_val=np.round(neg_scaled_map_loss.item(), 2), loss_type="map", shadow=False, title_add=title_add)
fig.savefig(f"{log_path}/posterior.png", bbox_inches='tight')
#fig.savefig(f"{log_path}/posterior.pgf", bbox_inches='tight')

In [ ]:
model.train()
likelihood.train()
model_MAP.train()
likelihood_MAP.train()

In [ ]:
pos_unscaled_MLL = -neg_scaled_mll_loss*len(*model.train_inputs)
pos_unscaled_MAP = -neg_scaled_map_loss*len(*model.train_inputs)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

bool_use_finite_difference_hessian = True

## Calculate the metrics
AIC_val, AIC_logs = AIC(pos_unscaled_MLL, num_params)
BIC_val, BIC_logs = BIC(pos_unscaled_MLL, num_params, torch.tensor(len(train_x)))
AIC_MAP_val, AIC_MAP_logs = AIC(pos_unscaled_MAP, num_params)
BIC_MAP_val, BIC_MAP_logs = BIC(pos_unscaled_MAP, num_params, torch.tensor(len(train_x)))
Lap0_val, Lap0_logs =     Laplace(model_MAP, pos_unscaled_MAP.clone(), uninformed=uninformed, use_finite_difference_hessian=bool_use_finite_difference_hessian, param_punish_term=0)
LapAIC_val, LapAIC_logs = Laplace(model_MAP, pos_unscaled_MAP.clone(), uninformed=uninformed, use_finite_difference_hessian=bool_use_finite_difference_hessian, param_punish_term=-1)
LapBIC_val, LapBIC_logs = Laplace(model_MAP, pos_unscaled_MAP.clone(), uninformed=uninformed, use_finite_difference_hessian=bool_use_finite_difference_hessian, param_punish_term="BIC")

In [ ]:
Lap0_logs["original symmetrized Hessian"]

In [ ]:
# TODO Check all the individual likelihood evaluations and see how they perform. Is the amount of change plausible compared to the optimum?
hessian_finite_differences_neg_unscaled_map = np.zeros((num_params, num_params))
for i, j in itertools.product(range(num_params), range(num_params)):
    h_i_vec = np.zeros(num_params)
    h_j_vec = np.zeros(num_params)
    h_i_vec[i] = 1.0
    h_j_vec[j] = 1.0
    hessian_finite_differences_neg_unscaled_map[i][j] = finite_difference_second_derivative_GP_neg_unscaled_map(model, likelihood, train_x, train_y, uninformed=uninformed, h_i_step=h_i_step, h_j_step=h_j_step, h_i_vec=h_i_vec, h_j_vec=h_j_vec)
hessian_finite_differences_neg_unscaled_map

In [ ]:
finite_difference_hessian(model_MAP, likelihood_MAP, num_params, train_x, train_y, uninformed=uninformed)

In [ ]:
model.covar_module(torch.tensor([train_x[0][0], train_x[1][0]])).evaluate()

In [ ]:
if np.isnan(Lap0_logs["laplace without replacement"].item()):
    # Load stop.ascii file and print it
    with open(f"stop.ascii", "r") as f:
        print(f.read())

In [ ]:
Lap0_logs

In [ ]:
print(torch.linalg.eig(Lap0_logs["original symmetrized Hessian"]))

In [ ]:
print(torch.linalg.eigh(Lap0_logs["original symmetrized Hessian"]))
print("---")
print(torch.linalg.det(Lap0_logs["original symmetrized Hessian"]))
print(torch.linalg.det(Lap0_logs["corrected Hessian"]))

In [ ]:
LapAIC_logs

In [ ]:
LapBIC_logs

In [ ]:
model_evidences = list()
model_evidence_logs = list()
uninformed = True

In [ ]:
maxcall = 1e+1
if maxcall in levels:
    print(maxcall)
    nested_pos_model_evidence_e1, Nested_logs_e1 = Nested(model, store_full=True, pickle_directory=os.path.join(log_path, "Nested_log"), maxcall=maxcall)
    model_evidences.append(nested_pos_model_evidence_e1)
    model_evidence_logs.append(Nested_logs_e1)

In [ ]:
maxcall = 1e+2
if maxcall in levels:
    print(maxcall)
    nested_pos_model_evidence_e2, Nested_logs_e2 = Nested(model, store_full=True, pickle_directory=os.path.join(log_path, "Nested_log"), maxcall=maxcall)
    model_evidences.append(nested_pos_model_evidence_e2)
    model_evidence_logs.append(Nested_logs_e2)

In [ ]:
maxcall = 1e+3
if maxcall in levels:
    print(maxcall)
    nested_pos_model_evidence_e3, Nested_logs_e3 = Nested(model, store_full=True, pickle_directory=os.path.join(log_path, "Nested_log"), maxcall=maxcall, uninformed=uninformed)
    model_evidences.append(nested_pos_model_evidence_e3)
    model_evidence_logs.append(Nested_logs_e3)

In [ ]:
maxcall = 1e+4
if maxcall in levels:
    print(maxcall)
    nested_pos_model_evidence_e4, Nested_logs_e4 = Nested(model, store_full=True, pickle_directory=os.path.join(log_path, "Nested_log"), maxcall=maxcall, uninformed=uninformed)
    model_evidences.append(nested_pos_model_evidence_e4)
    model_evidence_logs.append(Nested_logs_e4)

In [ ]:
maxcall = 1e+5
if maxcall in levels:
    print(maxcall)
    nested_pos_model_evidence_e5, Nested_logs_e5 = Nested(model, store_full=True, pickle_directory=os.path.join(log_path, "Nested_log"), maxcall=maxcall, uninformed=uninformed)
    model_evidences.append(nested_pos_model_evidence_e5)
    model_evidence_logs.append(Nested_logs_e5)

In [ ]:
for mod_log in model_evidence_logs:
    with open(f"{mod_log['res file']}", "rb") as f:
        res = dill.load(f)
    print(f"Num calls: {sum(res.ncall)}")
    print(f"logz Error: {res["logzerr"][-1]}")
    print(f"Accepted samples: {len(res.samples)}")

In [ ]:

plot_mll_opt = True
std_filter = None # float as 1e-3 or None
show_last_num = 0.1 # integer, float or None
filter_type = "none" # ("mean"), "max", "none"
xdim = 1
ydim = 2

param_names = [l[0] for l in list(model.named_parameters())]

f, axs = plt.subplots(1, len(levels), figsize=(4*len(levels), 4), sharey=True, layout="constrained")
if len(levels) == 1:
    axs = [axs]
for ax, level, model_evidence_log in zip(axs, levels, model_evidence_logs):
    if ax == axs[-1]:
        nested_sampling_plot(model, model_evidence_log, xdim = xdim, ydim = ydim, filter_type=filter_type, std_filter=std_filter, show_last_num=show_last_num, return_figure=False, title_add="", fig=f, ax=ax, display_figure=False, plot_mll_opt=plot_mll_opt, mll_opt_params=mll_opt_params, plot_lap=True, Lap0_logs=Lap0_logs, LapAIC_logs=LapAIC_logs, LapBIC_logs=LapBIC_logs, lap_colors = ["r", "pink", "white"], Lap_hess=Lap0_logs["original symmetrized Hessian"])
    else:
        nested_sampling_plot(model, model_evidence_log, xdim = xdim, ydim = ydim, filter_type=filter_type, std_filter=std_filter, show_last_num=show_last_num, return_figure=False, title_add="", fig=f, ax=ax, display_figure=False, plot_mll_opt=plot_mll_opt, mll_opt_params=mll_opt_params, plot_lap=True, Lap0_logs=Lap0_logs, LapAIC_logs=LapAIC_logs, LapBIC_logs=LapBIC_logs, lap_colors = ["r", "pink", "white"], Lap_hess=Lap0_logs["original symmetrized Hessian"])
    if ax == axs[0] or len(levels)==1:
        ax.set_ylabel(param_names[ydim])

In [ ]:
map_opt_params = [p.item() for p in model_MAP.parameters() if p.requires_grad]

plot_map_opt = False
std_filter = None # float as 1e-3 or None
show_last_num = 0.1 # integer, float or None
filter_type = "none" # ("mean"), "max", "none"
xdim = 1
ydim = 2

param_names = [l[0] for l in list(model.named_parameters())]

f, axs = plt.subplots(1, len(levels), figsize=(4*len(levels), 4), sharey=True, layout="constrained")
if len(levels) == 1:
    axs = [axs]
for ax, level, model_evidence_log in zip(axs, levels, model_evidence_logs):
    if ax == axs[-1]:
        posterior_surface_plot(model, model_evidence_log, xdim = xdim, ydim = ydim, filter_type=filter_type, std_filter=std_filter, show_last_num=show_last_num, return_figure=False, title_add="", fig=f, ax=ax, display_figure=False, plot_mll_opt=plot_map_opt, mll_opt_params=map_opt_params, plot_lap=True, Lap0_logs=Lap0_logs, LapAIC_logs=LapAIC_logs, LapBIC_logs=LapBIC_logs, lap_colors = ["r", "pink", "white"], uninformed=uninformed, Lap_hess=Lap0_logs["original symmetrized Hessian"])
    else:
        posterior_surface_plot(model, model_evidence_log, xdim = xdim, ydim = ydim, filter_type=filter_type, std_filter=std_filter, show_last_num=show_last_num, return_figure=False, title_add="", fig=f, ax=ax, display_figure=False, plot_mll_opt=plot_map_opt, mll_opt_params=map_opt_params, plot_lap=True, Lap0_logs=Lap0_logs, LapAIC_logs=LapAIC_logs, LapBIC_logs=LapBIC_logs, lap_colors = ["r", "pink", "white"], uninformed=uninformed, Lap_hess=Lap0_logs["original symmetrized Hessian"])
    if ax == axs[0] or len(levels)==1:
        ax.set_ylabel(param_names[ydim])

In [ ]:
# Find the best value and the corresponding hyperparameters
best_idx = np.argmax(res.logl)
best_hyperparameters = res.samples[best_idx]

In [ ]:
# Optimal GP according to Nested sampling
fixed_reinit(model, torch.tensor(best_hyperparameters))
model.eval()
likelihood.eval()
# mll of best parameterization according to Nested sampling:
mll_fkt = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
pos_scaled_nested_top_mll = (mll_fkt(model(train_x), train_y))*len(train_x)
pos_scaled_nested_top_map = pos_scaled_nested_top_mll+log_normalized_prior(model)*len(train_x)

fig = plt.figure(figsize=(16, 6))
ax = fig.add_subplot(1, 2, 1, projection='3d')

x_min=min(train_x[:,0])
x_max=max(train_x[:,0])
y_min=min(train_x[:,1])
y_max=max(train_x[:,1])


title_add = ""
if y_min == y_max:
    y_min = y_min-0.1
    y_max = y_max+0.1
    title_add += " Synthetic y stretch"
if x_min == x_max:
    x_min = x_min-0.1
    x_max = x_max+0.1
    title_add += " Synthetic x stretch"


fig, ax = plot_3d_gp(model, likelihood, data=torch.stack([train_x[:,0], train_x[:,1], train_y], -1), x_min=x_min, x_max=x_max, y_min=y_min, y_max=y_max, return_figure=True, fig=fig, ax=ax, display_figure=False, shadow=False)
#fig, ax = plot_model(model, likelihood, train_x, train_y, return_figure=True, loss_val=res.logl[best_idx], loss_type="Nested logl")
ax.title.set_text(f"{ax.title.get_text()} ; MLL: {pos_scaled_nested_top_mll.item():.2f}; MAP: {pos_scaled_nested_top_map.item():.2f}\n MLL (u) {-pos_scaled_nested_top_mll/len(train_x):.2f}; MAP (u) {-pos_scaled_nested_top_map/len(train_x):.2f}\n{title_add}")

In [ ]:
# Collect metric names and values into a list of tuples
metrics_data = [
    ("MLL", f"{pos_unscaled_MLL.item():.3f}"),
    ("MLL (loss)", f"{neg_scaled_mll_loss.item():.3f}"),
    ("AIC", f"{AIC_val.item():.3f}"),
    ("AIC (s)", f"{(AIC_val.item()*(-0.5)):.3f}"),
    ("BIC", f"{BIC_val.item():.3f}"),
    ("BIC (s)", f"{(BIC_val.item()*(-0.5)):.3f}"),
    

]
labels = [m[0] for m in metrics_data]
values = [m[1] for m in metrics_data]
# Print in transposed form
print("|        | " + " | ".join(labels) + " |")
print("|--------|" + "|".join(["-----------" for _ in labels]) + "|")
print("| Value  | " + " | ".join(values) + " |")
print()

metrics_data = [("MAP", f"{pos_unscaled_MAP.item():.3f}"),
    ("MAP (loss)", f"{neg_scaled_map_loss.item():.3f}"),
    ("AIC_M", f"{AIC_MAP_val.item():.3f}"),
    ("AIC_M (s)", f"{(AIC_MAP_val.item()*(-0.5)):.3f}"),
    ("BIC_M", f"{BIC_MAP_val.item():.3f}"),
    ("BIC_M (s)", f"{(BIC_MAP_val.item()*(-0.5)):.3f}"),
]

labels = [m[0] for m in metrics_data]
values = [m[1] for m in metrics_data]
# Print in transposed form
print("|        | " + " | ".join(labels) + " |")
print("|--------|" + "|".join(["-----------" for _ in labels]) + "|")
print("| Value  | " + " | ".join(values) + " |")
print()


metrics_data=[    
    ("Lap", f"{Lap0_logs["laplace without replacement"].item():.3f}"),
    ("Lap0", f"{Lap0_val.item():.3f}"),
    ("LapAIC", f"{LapAIC_val.item():.3f}"),
    ("LapBIC", f"{LapBIC_val.item():.3f}"),
    ]


labels = [m[0] for m in metrics_data]
values = [m[1] for m in metrics_data]
# Print in transposed form
print("|        | " + " | ".join(labels) + " |")
print("|--------|" + "|".join(["-----------" for _ in labels]) + "|")
print("| Value  | " + " | ".join(values) + " |")
print()

metrics_data = [(f"Nested ({level:.1e})", f"{Nested_val:.3f}") 
                for level, Nested_val in zip(levels, model_evidences)]

# Transpose the table: one row for the label, another for the value
labels = [m[0] for m in metrics_data]
values = [m[1] for m in metrics_data]

# Print in transposed form
print("|        | " + " | ".join(labels) + " |")
print("|--------|" + "|".join(["-----------" for _ in labels]) + "|")
print("| Value  | " + " | ".join(values) + " |")

In [ ]:
print(f"{dataset_name} - {pattern} {X1_noise};{X2_noise} - {kernel_name}")


In [ ]:

fixed_reinit(model_MAP, torch.tensor(best_hyperparameters))
mll_fkt = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood_MAP, model_MAP)
neg_scaled_map_loss = -mll_fkt(model_MAP(train_x), train_y)-log_normalized_prior(model_MAP)
pos_unscaled_MAP = -neg_scaled_map_loss*len(*model.train_inputs)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
Nested_Lap0_val, Nested_Lap0_logs = Laplace(model_MAP, pos_unscaled_MAP, param_punish_term=0)

print(f"Lap/nested = {Lap0_logs["laplace without replacement"].item()/nested_pos_model_evidence_e5:.3f}"),
print(f"Lap0/nested = {Lap0_val/nested_pos_model_evidence_e5:.3f}")
print(f"AIC (s)/nested = {AIC_val*(-0.5)/nested_pos_model_evidence_e5:.3f}") 
print(f"nested Lap0/nested = {Nested_Lap0_val/nested_pos_model_evidence_e5:.3f}\n")
print("```python")
print(Nested_Lap0_val)

print(Nested_Lap0_logs)
print("```")

# Temp area

In [ ]:
one_dim_data

In [ ]:
two_dim_data

In [ ]:
print(one_dim_data[0] - two_dim_data[0][:,0])
one_dim_data[1] - two_dim_data[1]

In [ ]:
one_dim_mean_vec - two_dim_mean_vec

In [ ]:
np.min(np.array((one_dim_cov_matr - two_dim_cov_matr).detach().numpy()))
# The value is negative (-0.001...), hence the two_dim_cov_matr is larger
# This can happen when either the noise is larger in the two-dim case or the lengthscale is larger
# Here, the noise is insignificantly smaller in the 2D case and the lengthscale is just slightly larger

# HOWEVER, once I set the parameter values to be equal to the trained parameters of the 2D case, the difference between the covariance matrices is reduced to -4e-5, which could be argued is just numerical error.



In [ ]:
# Torch softplus call
print(torch.nn.functional.softplus(torch.tensor(-11.6140)) - torch.nn.functional.softplus(torch.tensor(-11.7261)))
print(torch.nn.functional.softplus(torch.tensor(-0.3335)) - torch.nn.functional.softplus(torch.tensor(-0.3326)))
print(torch.nn.functional.softplus(torch.tensor(-0.3335)))
print(torch.nn.functional.softplus(torch.tensor(-0.3326)))
# Given the insignificant noise difference I would conclude the issue is with the lengthscale values

In [ ]:
print(one_dim_post_mean_vec)
print(one_dim_post_cov_matr)

In [ ]:
print(two_dim_post_mean_vec)
print(two_dim_post_cov_matr)

In [ ]:
# the differences
print(one_dim_post_mean_vec - two_dim_post_mean_vec)
print(one_dim_post_cov_matr - two_dim_post_cov_matr)

In [ ]:
print(np.max(np.array((one_dim_post_mean_vec - two_dim_post_mean_vec).detach().numpy())))
np.max(np.array((one_dim_post_cov_matr - two_dim_post_cov_matr).detach().numpy()))


In [ ]:
for one_d, two_d in zip((one_dim_train_log.neg_loss[0], one_dim_train_log.x[0]), (two_dim_train_log.neg_loss[0], two_dim_train_log.x[0])):
    #print(f"{one_d[0]} - {two_d[0]}")
    print(one_d[0] - two_d[0])
    print(one_d[1] - two_d[1])

In [ ]:
print("1D", len(one_dim_train_log.neg_loss[0]))
print("2D", len(two_dim_train_log.neg_loss[0]))

In [ ]:
for i in range(max((len(one_dim_train_log.neg_loss[0]), len(two_dim_train_log.neg_loss[0])))):
    print(f"Step {i}")
    try:
        one_dim_params = one_dim_train_log.x[0][i]
        one_dim_MAP = one_dim_train_log.neg_loss[0][i]
    except:
        one_dim_params = None
        one_dim_MAP = None
    try:
        two_dim_params = two_dim_train_log.x[0][i]
        two_dim_MAP = two_dim_train_log.neg_loss[0][i]
    except:
        two_dim_params = None
        two_dim_MAP = None

    if one_dim_params is not None and two_dim_params is not None:
        for one_d, two_d in zip(one_dim_params, two_dim_params):
            print(f"{one_d} - {two_d}")
        print(f"MAPs {one_dim_MAP} - {two_dim_MAP}")
        print(f"MAP diff {one_dim_MAP - two_dim_MAP}")
    elif one_dim_params is not None:
        print(f"One dim params {one_dim_params} - {one_dim_MAP}")
    elif two_dim_params is not None:
        print(f"Two dim params {two_dim_params} - {two_dim_MAP}")


## 1D Block that performs a grid search over parameters and stores the MLL/MAP values (+ first and second derivative?)


In [ ]:
!pip install tqdm

In [ ]:
likelihood_exploration_df = pd.read_csv("likelihood_exploration_grid.csv", sep=";")

In [ ]:
like_explor_positions = likelihood_exploration_df[["log noise", "log lengthscale 1", "log lengthscale 2"]].to_numpy()
like_explor_pos_scaled_mlls = likelihood_exploration_df[["mll"]].to_numpy().flatten()
like_explor_pos_scaled_maps = likelihood_exploration_df[["map"]].to_numpy().flatten()
like_explor_pos_scaled_maps_hess = likelihood_exploration_df["d2/dx2 map"].apply(lambda x: torch.tensor([float(f) for f in re.findall(r"[-+]?\d*\.\d*", x)]).reshape(3,3).numpy())
like_explor_pos_scaled_maps_hess_dets = likelihood_exploration_df["d2/dx2 map"].apply(lambda x: torch.linalg.det(torch.tensor([float(f) for f in re.findall(r"[-+]?\d*\.\d*", x)]).reshape(3,3)).numpy())
#like_explor_mlls_hess = likelihood_exploration_df["d2/dx2 mll"]

In [ ]:
print("Optimal parameters")
print("MAP: ", like_explor_positions[np.argmax(like_explor_pos_scaled_maps)], ":", max(like_explor_pos_scaled_maps))
print("MLL: ", like_explor_positions[np.argmax(like_explor_pos_scaled_mlls)], ":", max(like_explor_pos_scaled_mlls))

In [ ]:
diag_neg_truth_map = [h[1][1]<0 and h[2][2]<0 for h in like_explor_pos_scaled_maps_hess]

In [ ]:
sum(diag_neg_truth_map)

In [ ]:
sum([d < 0 for d in like_explor_pos_scaled_maps_hess_dets])

In [ ]:

# Count instances where the Hessian is not symmetric
non_sym_count = sum([torch.allclose(h, h.T, atol=1e-5) for h in torch.tensor(like_explor_pos_scaled_maps_hess)])
print(f"Number of non-symmetric Hessians: {len(like_explor_pos_scaled_maps_hess) - non_sym_count}")

In [ ]:
like_explor_pos_scaled_maps_hess[305353]

In [ ]:
max(like_explor_pos_scaled_maps_hess_dets)


In [ ]:
like_explor_pos_scaled_maps_hess[np.argmax(like_explor_pos_scaled_maps_hess_dets)]

In [ ]:



xdim = 0
ydim = 2

ax_names = [["Log Noise", "Log Lengthscale 1", "Log Lengthscale 2"][xdim], ["Log Noise", "Log Lengthscale 1", "Log Lengthscale 2"][ydim]]


scatter_function_visualization(like_explor_positions, like_explor_pos_scaled_maps, axis_names=ax_names, show_best=True, filter_mode="max", std_filter=10, display_figure=True, xdim=xdim, ydim=ydim)

In [ ]:



xdim = 0
ydim = 2

ax_names = [["Log Noise", "Log Lengthscale 1", "Log Lengthscale 2"][xdim], ["Log Noise", "Log Lengthscale 1", "Log Lengthscale 2"][ydim]]


scatter_function_visualization(like_explor_positions, like_explor_pos_scaled_maps_hess_dets, axis_names=ax_names, show_best=True, filter_mode="max", std_filter=10, display_figure=True, xdim=xdim, ydim=ydim)

## 2D Block that performs a grid search over parameters and stores the MLL/MAP values (+ first and second derivative?)


In [ ]:
!pip install tqdm

In [ ]:
likelihood_exploration_df_2D = pd.read_csv("likelihood_exploration_grid_2D.csv", sep=";")

In [ ]:
like_explor_positions_2D = likelihood_exploration_df_2D[["log noise", "log lengthscale 1", "log lengthscale 2"]].to_numpy()
like_explor_pos_scaled_mlls_2D = likelihood_exploration_df_2D[["mll"]].to_numpy().flatten()
like_explor_pos_scaled_maps_2D = likelihood_exploration_df_2D[["map"]].to_numpy().flatten()
like_explor_pos_scaled_maps_hess_2D = likelihood_exploration_df_2D["d2/dx2 map"].apply(lambda x: torch.tensor([float(f) for f in re.findall(r"[-+]?\d*\.\d*", x)]).reshape(3,3).numpy())
like_explor_pos_scaled_maps_hess_dets_2D = likelihood_exploration_df_2D["d2/dx2 map"].apply(lambda x: torch.linalg.det(torch.tensor([float(f) for f in re.findall(r"[-+]?\d*\.\d*", x)]).reshape(3,3)).numpy())
#like_explor_mlls_hess = likelihood_exploration_df["d2/dx2 mll"]

In [ ]:
print("Optimal parameters")
print("MAP: ", like_explor_positions_2D[np.argmax(like_explor_pos_scaled_maps_2D)], ":", max(like_explor_pos_scaled_maps_2D))
print("MLL: ", like_explor_positions_2D[np.argmax(like_explor_pos_scaled_mlls_2D)], ":", max(like_explor_pos_scaled_mlls_2D))

In [ ]:
diag_neg_truth_map_2D = [h[1][1]<0 and h[2][2]<0 for h in like_explor_pos_scaled_maps_hess_2D]

In [ ]:
sum(diag_neg_truth_map_2D)

In [ ]:
sum([d < 0 for d in like_explor_pos_scaled_maps_hess_dets_2D])

In [ ]:
# Count instances where the Hessian is not symmetric
non_sym_count_2D = sum([torch.allclose(h, h.T, atol=1e-5) for h in torch.tensor(like_explor_pos_scaled_maps_hess_2D)])
print(f"Number of non-symmetric Hessians: {len(like_explor_pos_scaled_maps_hess) - non_sym_count_2D}")

In [ ]:
max(like_explor_pos_scaled_maps_hess_dets_2D)


In [ ]:
like_explor_pos_scaled_maps_hess_2D[np.argmax(like_explor_pos_scaled_maps_hess_dets_2D)]

In [ ]:



xdim = 0
ydim = 2

ax_names = [["Log Noise", "Log Lengthscale 1", "Log Lengthscale 2"][xdim], ["Log Noise", "Log Lengthscale 1", "Log Lengthscale 2"][ydim]]


scatter_function_visualization(like_explor_positions_2D, like_explor_pos_scaled_maps_2D, axis_names=ax_names, show_best=True, filter_mode="max", std_filter=10, display_figure=True, xdim=xdim, ydim=ydim)

In [ ]:



xdim = 0
ydim = 2

ax_names = [["Log Noise", "Log Lengthscale 1", "Log Lengthscale 2"][xdim], ["Log Noise", "Log Lengthscale 1", "Log Lengthscale 2"][ydim]]


scatter_function_visualization(like_explor_positions_2D, like_explor_pos_scaled_maps_hess_dets_2D, axis_names=ax_names, show_best=True, filter_mode="max", std_filter=10, display_figure=True, xdim=xdim, ydim=ydim)

In [ ]:
print("1D det argmax", np.argmax(like_explor_pos_scaled_maps_hess_dets))
print("2D det argmax", np.argmax(like_explor_pos_scaled_maps_hess_dets_2D))

In [ ]:
print(like_explor_pos_scaled_maps_hess_dets[np.argmax(like_explor_pos_scaled_maps_hess_dets)])
like_explor_pos_scaled_maps_hess_dets_2D[np.argmax(like_explor_pos_scaled_maps_hess_dets)]

In [ ]:
print(like_explor_pos_scaled_maps_hess_dets_2D[np.argmax(like_explor_pos_scaled_maps_hess_dets_2D)])
like_explor_pos_scaled_maps_hess_dets[np.argmax(like_explor_pos_scaled_maps_hess_dets_2D)]

In [ ]:
import random
index = random.randint(0, len(like_explor_pos_scaled_maps_hess_2D)-1)
index = 204949
print(index)
print(like_explor_positions[index])
print(like_explor_positions_2D[index])
print(like_explor_pos_scaled_maps_hess[index])
print(like_explor_pos_scaled_maps_hess_2D[index])
print(like_explor_pos_scaled_maps_hess[index] - like_explor_pos_scaled_maps_hess_2D[index])
#print(torch.linalg.det(like_explor_maps_hess[index] - like_explor_maps_hess_2D[index]))

In [ ]:
f_minus = 204948
f = 204949
f_plus = 204950

print((like_explor_pos_scaled_maps[f_minus] - 2*like_explor_pos_scaled_maps[f] + like_explor_pos_scaled_maps[f_plus]) / ((-0.404)**2))
print((like_explor_pos_scaled_maps_2D[f_minus] - 2*like_explor_pos_scaled_maps_2D[f] + like_explor_pos_scaled_maps_2D[f_plus]) / ((-0.404)**2))
 

In [ ]:
# The MAP and MLL optima are at the same location in both GPs, at least
index = np.argmax(like_explor_pos_scaled_maps)
print(index)
print(like_explor_positions[index])
print(like_explor_positions_2D[index])
print(like_explor_pos_scaled_maps_hess[index])
print(like_explor_pos_scaled_maps_hess_2D[index])
print(like_explor_pos_scaled_maps_hess[index] - like_explor_pos_scaled_maps_hess_2D[index])

In [ ]:
like_explor_maps_hess_difference_dets = np.array([np.linalg.det(p) for p in (like_explor_pos_scaled_maps_hess - like_explor_pos_scaled_maps_hess_2D).to_numpy()])

In [ ]:



xdim = 0
ydim = 2

ax_names = [["Log Noise", "Log Lengthscale 1", "Log Lengthscale 2"][xdim], ["Log Noise", "Log Lengthscale 1", "Log Lengthscale 2"][ydim]]


scatter_function_visualization(like_explor_positions_2D, like_explor_maps_hess_difference_dets, axis_names=ax_names, show_best=True, filter_mode="none", std_filter=100, display_figure=True, xdim=xdim, ydim=ydim)